<h2> Imports </h2>

In [1]:
import pandas as pd
import numpy as np
import json
import os
import math
import matplotlib.pyplot as plt
from typing import Dict, Any, Tuple

from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import KMeans

c:\Users\niush\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\niush\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
c:\Users\niush\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\niush\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<h1> Step 1: Prepare data and configure the environment </h1>

<h3> 1.1. Data Loading (Data I/O) </h3>

In [2]:
# Define the base directories
dataset_dir = '../Data_Generator/datasets'
topology_dir = '../Topology_Generator/topologies'

Loading dataset

In [3]:
# Global container
datasets = {}

def load_datasets_from_directory(dataset_dir, verbose=True):
    """
    Build 'episode-first' structure:
    datasets = {
        "ep_000": {
            "light":   { "episodes": df, "agents": df, "arrivals": df, "tasks": df },
            "moderate":{ ... },
            "heavy":   { ... }
        },
        "ep_001": { ... },
        ...
    }
    """
    # Step 1 — detect scenarios (light/moderate/heavy/...)
    scenarios = [
        name for name in os.listdir(dataset_dir)
        if os.path.isdir(os.path.join(dataset_dir, name))
    ]

    # Step 2 — load per scenario and per episode (ep_XXX)
    scenario_to_episodes = {}
    for scenario in scenarios:
        scn_path = os.path.join(dataset_dir, scenario)
        ep_dirs = sorted([
            ep for ep in os.listdir(scn_path)
            if os.path.isdir(os.path.join(scn_path, ep)) and ep.startswith("ep_")
        ])
        if not ep_dirs and verbose:
            print(f"[warn] no ep_* folders found under scenario '{scenario}'")

        scenario_to_episodes[scenario] = {}
        for ep_name in ep_dirs:
            ep_path = os.path.join(scn_path, ep_name)
            try:
                scenario_to_episodes[scenario][ep_name] = {
                    "episodes": pd.read_csv(os.path.join(ep_path, "episodes.csv")),
                    "agents":   pd.read_csv(os.path.join(ep_path, "agents.csv")),
                    "arrivals": pd.read_csv(os.path.join(ep_path, "arrivals.csv")),
                    "tasks":    pd.read_csv(os.path.join(ep_path, "tasks.csv")),
                }
            except FileNotFoundError as e:
                if verbose:
                    print(f"[error] missing CSV in {ep_path}: {e}")
                continue

    # Step 3 — invert structure: episodes → scenarios
    datasets.clear()
    for scenario, eps in scenario_to_episodes.items():
        for ep_name, dfs in eps.items():
            if ep_name not in datasets:
                datasets[ep_name] = {}
            datasets[ep_name][scenario] = dfs

    # Optional summary printing
    if verbose:
        print("=== Dataset Summary (episode-first) ===")
        print(f"episodes: {len(datasets)}  | scenarios detected: {len(scenarios)} -> {sorted(scenarios)}")
        for ep_name in sorted(datasets.keys()):
            scenarios_here = sorted(datasets[ep_name].keys())
            print(f"  - {ep_name}: scenarios = {scenarios_here}")
            for scn in scenarios_here:
                dfs = datasets[ep_name][scn]
                n_ep   = len(dfs['episodes'])
                n_ag   = len(dfs['agents'])
                n_arr  = len(dfs['arrivals'])
                n_task = len(dfs['tasks'])
                print(f"      {scn:9s} → episodes:{n_ep:3d}  agents:{n_ag:4d}  arrivals:{n_arr:6d}  tasks:{n_task:6d}")
        print("=======================================")

    return datasets

In [4]:
# ---- load all datasets (episode-first) ----
datasets = load_datasets_from_directory(dataset_dir, verbose=True)

# ---- choose an episode and a scenario for printing ----
# pick first available episode if you don't want to hardcode
ep_name = sorted(datasets.keys())[0] if datasets else None
scenario = "heavy"  # you can change to "light"/"moderate" if needed

if ep_name is not None and scenario in datasets[ep_name]:
    print(f"\n[info] printing from episode='{ep_name}', scenario='{scenario}'")

    print("\nagents:")
    display(datasets[ep_name][scenario]['agents'].head())
    datasets[ep_name][scenario]['agents'].info()

    print("\narrivals:")
    display(datasets[ep_name][scenario]['arrivals'].head())
    datasets[ep_name][scenario]['arrivals'].info()

    print("\nepisodes:")
    display(datasets[ep_name][scenario]['episodes'].head())
    datasets[ep_name][scenario]['episodes'].info()

    print("\ntasks:")
    display(datasets[ep_name][scenario]['tasks'].head())
    datasets[ep_name][scenario]['tasks'].info()
else:
    print("[error] no datasets found or requested scenario is missing for the chosen episode.")

=== Dataset Summary (episode-first) ===
episodes: 1  | scenarios detected: 3 -> ['heavy', 'light', 'moderate']
  - ep_000: scenarios = ['heavy', 'light', 'moderate']
      heavy     → episodes:  1  agents:  18  arrivals: 30636  tasks: 30636
      light     → episodes:  1  agents:  18  arrivals:  2113  tasks:  2113
      moderate  → episodes:  1  agents:  18  arrivals:  8262  tasks:  8262

[info] printing from episode='ep_000', scenario='heavy'

agents:


,agent_id,f_local,m_local,lam_sec
0,0,1.741183e+09,5713.849721,0.708673
1,1,1.352326e+09,4566.428755,0.234989
2,2,1.726668e+09,5815.120004,0.228174
3,3,1.543616e+09,3539.850245,0.310369
4,4,1.130883e+09,4161.367769,0.548990


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   agent_id  18 non-null     int64  
 1   f_local   18 non-null     float64
 2   m_local   18 non-null     float64
 3   lam_sec   18 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 704.0 bytes

arrivals:


,scenario,episode_id,t_slot,t_time,agent_id,task_id
0,heavy,0,0,0.0,0,0
1,heavy,0,0,0.0,1,1
2,heavy,0,0,0.0,4,2
3,heavy,0,0,0.0,7,3
4,heavy,0,0,0.0,10,4


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    30636 non-null  object 
 1   episode_id  30636 non-null  int64  
 2   t_slot      30636 non-null  int64  
 3   t_time      30636 non-null  float64
 4   agent_id    30636 non-null  int64  
 5   task_id     30636 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 1.4+ MB

episodes:


,scenario,episode_id,Delta,T_slots,hours,N_agents,seed
0,heavy,0,1.0,3600,1.0,18,345


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   scenario    1 non-null      object 
 1   episode_id  1 non-null      int64  
 2   Delta       1 non-null      float64
 3   T_slots     1 non-null      int64  
 4   hours       1 non-null      float64
 5   N_agents    1 non-null      int64  
 6   seed        1 non-null      int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 184.0+ bytes

tasks:


,scenario,episode_id,task_id,agent_id,t_arrival_slot,t_arrival_time,b_mb,rho_cyc_per_mb,c_cycles,mem_mb,modality,has_deadline,deadline_s,deadline_time,non_atomic,split_ratio,action_space_hint
0,heavy,0,0,0,0,0.0,7.202096,9.727147e+08,7.005585e+09,66.611010,sensor,1,0.800726,0.800726,0,0.000000,discrete
1,heavy,0,1,1,0,0.0,5.479984,1.314973e+09,7.206031e+09,77.928800,image,1,0.615113,0.615113,0,0.000000,discrete
2,heavy,0,2,4,0,0.0,8.421977,2.500222e+09,2.105681e+10,72.966446,text,1,0.323007,0.323007,1,0.539704,continuous
3,heavy,0,3,7,0,0.0,6.324986,1.779582e+09,1.125583e+10,56.492900,sensor,1,0.481587,0.481587,0,0.000000,discrete
4,heavy,0,4,10,0,0.0,11.473269,1.087572e+09,1.247800e+10,73.389854,sensor,1,0.594564,0.594564,0,0.000000,discrete


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   scenario           30636 non-null  object 
 1   episode_id         30636 non-null  int64  
 2   task_id            30636 non-null  int64  
 3   agent_id           30636 non-null  int64  
 4   t_arrival_slot     30636 non-null  int64  
 5   t_arrival_time     30636 non-null  float64
 6   b_mb               30636 non-null  float64
 7   rho_cyc_per_mb     30636 non-null  float64
 8   c_cycles           30636 non-null  float64
 9   mem_mb             30636 non-null  float64
 10  modality           30636 non-null  object 
 11  has_deadline       30636 non-null  int64  
 12  deadline_s         10673 non-null  float64
 13  deadline_time      10673 non-null  float64
 14  non_atomic         30636 non-null  int64  
 15  split_ratio        30636 non-null  float64
 16  action_space_hint  306

Loading topology

In [5]:
# Global container
topologies = {}

def load_topologies_from_directory(topology_dir):
    
    for topology_name in os.listdir(topology_dir):
        topology_path = os.path.join(topology_dir, topology_name)
        
        # Only process directories
        if os.path.isdir(topology_path):
            topology_json_path = os.path.join(topology_path, "topology.json")
            meta_json_path = os.path.join(topology_path, "topology_meta.json")
            connection_matrix_csv_path = os.path.join(topology_path, "connection_matrix.csv")
            
             # --- Load JSON & CSV files ---
            topology_data = None
            meta_data = None
            with open(topology_json_path, "r", encoding="utf-8") as f:
                topology_data = json.load(f)
            with open(meta_json_path, "r", encoding="utf-8") as f:
                meta_data = json.load(f)
            
            # The first column is just for displaying row names, not part of the capacity matrix. 
            # So the best way is to index the first column. (index_col=0)
            connection_matrix = pd.read_csv(connection_matrix_csv_path, index_col=0)
            
            # Store the topology details and the loaded CSV
            topologies[topology_name] = {
                "topology_data": topology_data,
                "meta_data": meta_data,
                "connection_matrix": connection_matrix  # Store the loaded CSV data
            }

In [6]:
load_topologies_from_directory(topology_dir)

print('topology clustered -> connection_matrix')
display(topologies['clustered']['connection_matrix'].head())
topologies['clustered']['connection_matrix'].info()

print('\ntopology clustered -> topology_data')
print(topologies['clustered']['topology_data'])

print('\ntopology clustered -> meta_data')
print(topologies['clustered']['meta_data'])

topology clustered -> connection_matrix


,mec_0,mec_1,mec_2,mec_3,mec_4,mec_5,mec_6,mec_7,mec_8,mec_9,mec_10,mec_11,mec_12,mec_13,mec_14,mec_15,mec_16,mec_17,cloud
mec_0,0.000000,9.798801,11.958295,11.470404,9.980195,8.814050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.912800
mec_1,9.798801,0.000000,8.375143,11.535600,10.702584,9.136893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,104.609202
mec_2,11.958295,8.375143,0.000000,8.814632,9.637714,9.170700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,92.055348
mec_3,11.470404,11.535600,8.814632,0.000000,10.377054,11.920252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84.851664
mec_4,9.980195,10.702584,9.637714,10.377054,0.000000,8.105383,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.076301


<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, mec_0 to mec_17
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mec_0   18 non-null     float64
 1   mec_1   18 non-null     float64
 2   mec_2   18 non-null     float64
 3   mec_3   18 non-null     float64
 4   mec_4   18 non-null     float64
 5   mec_5   18 non-null     float64
 6   mec_6   18 non-null     float64
 7   mec_7   18 non-null     float64
 8   mec_8   18 non-null     float64
 9   mec_9   18 non-null     float64
 10  mec_10  18 non-null     float64
 11  mec_11  18 non-null     float64
 12  mec_12  18 non-null     float64
 13  mec_13  18 non-null     float64
 14  mec_14  18 non-null     float64
 15  mec_15  18 non-null     float64
 16  mec_16  18 non-null     float64
 17  mec_17  18 non-null     float64
 18  cloud   18 non-null     float64
dtypes: float64(19)
memory usage: 2.8+ KB

topology clustered -> topology_data
{'number_of_servers': 18, 'private_cpu

<h3> 1.2. Data Validation </h3>

Before using the data, we must validate that required columns exist and that IDs match properly.

**The code below performs three layers of checks:** 

- Validate each dataset (episodes/agents/arrivals/tasks)
- Validate each topology (JSON and connection matrix)
- Validate dataset–topology pairs for unit alignment and overall consistency

In [7]:
# ---------- Generic helpers ----------
def _require(cond: bool, msg: str, errors: list):
    # Collect errors instead of stopping at first failure
    if not cond:
        errors.append(msg)

def _has_cols(df: pd.DataFrame, cols: list) -> bool:
    return all(c in df.columns for c in cols)

# ---------- Dataset-level validation ----------
def validate_one_dataset(dataset_key: str, ds: dict) -> list:
    """
    Validate a single dataset pack (episodes/agents/arrivals/tasks) for one (episode, scenario).
    'dataset_key' is just a label for error messages, e.g. 'ep_000/heavy'.
    """
    errors = []
    episodes = ds.get("episodes")
    agents   = ds.get("agents")
    arrivals = ds.get("arrivals")
    tasks    = ds.get("tasks")

    # 1) Presence checks
    _require(isinstance(episodes, pd.DataFrame), f"[{dataset_key}] episodes missing or not a DataFrame", errors)
    _require(isinstance(agents,   pd.DataFrame), f"[{dataset_key}] agents missing or not a DataFrame", errors)
    _require(isinstance(arrivals, pd.DataFrame), f"[{dataset_key}] arrivals missing or not a DataFrame", errors)
    _require(isinstance(tasks,    pd.DataFrame), f"[{dataset_key}] tasks missing or not a DataFrame", errors)
    if errors:
        return errors

    # 2) Required columns (based on your generators)
    req_ep_cols  = ["scenario", "episode_id", "Delta", "T_slots", "hours", "N_agents", "seed"]
    req_ag_cols  = ["agent_id", "f_local", "m_local", "lam_sec"]
    req_ar_cols  = ["scenario", "episode_id", "t_slot", "t_time", "agent_id", "task_id"]
    req_tk_cols  = [
        "scenario","episode_id","task_id","agent_id","t_arrival_slot","t_arrival_time",
        "b_mb","rho_cyc_per_mb","c_cycles","mem_mb","modality",
        "has_deadline","deadline_s","deadline_time","non_atomic","split_ratio","action_space_hint"
    ]
    _require(_has_cols(episodes, req_ep_cols), f"[{dataset_key}] episodes missing required columns", errors)
    _require(_has_cols(agents,   req_ag_cols), f"[{dataset_key}] agents missing required columns", errors)
    _require(_has_cols(arrivals, req_ar_cols), f"[{dataset_key}] arrivals missing required columns", errors)
    _require(_has_cols(tasks,    req_tk_cols), f"[{dataset_key}] tasks missing required columns", errors)
    if errors:
        return errors

    # 3) Integrity checks
    # unique task_id
    _require(tasks["task_id"].is_unique, f"[{dataset_key}] task_id is not unique", errors)

    # agent id range & count vs episodes.N_agents
    if len(agents):
        min_id = agents["agent_id"].min()
        max_id = agents["agent_id"].max()
        expected_n = int(episodes["N_agents"].iloc[0])
        _require(min_id == 0, f"[{dataset_key}] agent_id should start at 0 (got {min_id})", errors)
        _require(max_id == expected_n - 1,
                 f"[{dataset_key}] agent_id max should be N_agents-1 ({expected_n-1}), got {max_id}", errors)

    # cross refs
    valid_agents = set(agents["agent_id"].tolist())
    bad_arr_agents = set(arrivals["agent_id"]) - valid_agents
    bad_task_agents = set(tasks["agent_id"]) - valid_agents
    _require(len(bad_arr_agents) == 0, f"[{dataset_key}] arrivals contain unknown agent_id(s): {sorted(bad_arr_agents)}", errors)
    _require(len(bad_task_agents) == 0, f"[{dataset_key}] tasks contain unknown agent_id(s): {sorted(bad_task_agents)}", errors)

    # non-negative task numerics
    for col in ["b_mb","rho_cyc_per_mb","c_cycles","mem_mb"]:
        if col in tasks.columns:
            _require((tasks[col] >= 0).all(), f"[{dataset_key}] tasks.{col} has negative values", errors)

    # deadline coherence
    if "has_deadline" in tasks.columns and "deadline_s" in tasks.columns:
        bad_deadline = tasks[(tasks["has_deadline"] == 1) & ((tasks["deadline_s"].isna()) | (tasks["deadline_s"] <= 0))]
        _require(len(bad_deadline) == 0, f"[{dataset_key}] tasks with deadline have invalid deadline_s", errors)

    # single Delta / T_slots inside this (episode, scenario)
    _require(episodes["Delta"].nunique() == 1, f"[{dataset_key}] multiple Delta values in episodes", errors)
    _require(episodes["T_slots"].nunique() == 1, f"[{dataset_key}] multiple T_slots in episodes", errors)

    # arrivals inside range
    T_slots = int(episodes["T_slots"].iloc[0])
    _require(int(tasks["t_arrival_slot"].max()) <= T_slots - 1,
             f"[{dataset_key}] t_arrival_slot exceeds T_slots-1", errors)

    return errors

# ---------- Topology-level validation ----------
def validate_one_topology(topology_name: str, topo_entry: dict) -> list:
    errors = []
    topo = topo_entry.get("topology_data")
    meta = topo_entry.get("meta_data")
    Mdf  = topo_entry.get("connection_matrix")

    _require(isinstance(topo, dict), f"[{topology_name}] topology_data missing or not a dict", errors)
    _require(isinstance(meta, dict), f"[{topology_name}] meta_data missing or not a dict", errors)
    _require(isinstance(Mdf,  pd.DataFrame), f"[{topology_name}] connection_matrix CSV missing or not a DataFrame", errors)
    if errors:
        return errors

    req_keys = [
        "number_of_servers","private_cpu_capacities","public_cpu_capacities",
        "cloud_computational_capacity","connection_matrix","time_step"
    ]
    for k in req_keys:
        _require(k in topo, f"[{topology_name}] topology.json missing key: {k}", errors)
    if errors:
        return errors

    K = int(topo["number_of_servers"])
    _require(len(topo["private_cpu_capacities"]) == K, f"[{topology_name}] private_cpu_capacities length != K", errors)
    _require(len(topo["public_cpu_capacities"])  == K, f"[{topology_name}] public_cpu_capacities length != K", errors)

    Mjson = topo["connection_matrix"]
    _require(isinstance(Mjson, list) and len(Mjson) == K and (K == 0 or len(Mjson[0]) == K+1),
             f"[{topology_name}] connection_matrix in JSON must be K x (K+1)", errors)
    _require(Mdf.shape == (K, K+1), f"[{topology_name}] connection_matrix.csv shape must be K x (K+1)", errors)

    vert_csv = Mdf.iloc[:, K]
    _require((vert_csv > 0).all(), f"[{topology_name}] MEC->Cloud capacities must be > 0", errors)
    horiz_csv = Mdf.iloc[:, :K]
    _require((horiz_csv.values >= 0).all(), f"[{topology_name}] MEC<->MEC capacities contain negatives", errors)

    _require("time_step" in topo, f"[{topology_name}] missing time_step", errors)
    return errors

# ---------- Pairwise validation (dataset <-> topology) ----------
def validate_dataset_topology_pair(ep_name: str, scenario: str, ds: dict,
                                   topology_name: str, topo_entry: dict) -> list:
    """
    Validate alignment between one (episode, scenario) dataset and one topology.
    Ensures Delta == time_step and basic feasibility checks.
    """
    errors = []
    episodes = ds["episodes"]
    topo     = topo_entry["topology_data"]
    K        = int(topo["number_of_servers"])

    # Delta vs time_step
    Delta = float(episodes["Delta"].iloc[0])
    time_step = float(topo["time_step"])
    _require(abs(Delta - time_step) < 1e-9,
             f"[{ep_name}/{scenario} x {topology_name}] Delta ({Delta}) != time_step ({time_step})", errors)

    # Non-negative compute capacities
    priv = topo["private_cpu_capacities"]
    pub  = topo["public_cpu_capacities"]
    cloud = topo["cloud_computational_capacity"]
    _require(all(x >= 0 for x in priv) and all(x >= 0 for x in pub) and cloud >= 0,
             f"[{ep_name}/{scenario} x {topology_name}] negative compute capacities detected", errors)

    # Simple agent→MEC mapping (modulo) is within bounds
    N_agents = int(episodes["N_agents"].iloc[0])
    mapped = [(aid % K) for aid in range(N_agents)] if K > 0 else []
    _require(all(0 <= m < K for m in mapped) if mapped else True,
             f"[{ep_name}/{scenario} x {topology_name}] agent->MEC mapping out of bounds", errors)

    return errors

# ---------- Episode-level Delta consistency across scenarios ----------
def validate_episode_delta_consistency(ep_name: str, ep_dict: dict) -> list:
    """
    Check that all scenarios inside one episode share the same Delta and T_slots.
    """
    errors = []
    deltas = set()
    tslots = set()
    for scenario, ds in ep_dict.items():
        ep_df = ds["episodes"]
        if len(ep_df):
            deltas.add(float(ep_df["Delta"].iloc[0]))
            tslots.add(int(ep_df["T_slots"].iloc[0]))
        else:
            errors.append(f"[{ep_name}/{scenario}] episodes.csv is empty")
    if len(deltas) > 1:
        errors.append(f"[{ep_name}] multiple Delta values across scenarios: {sorted(deltas)}")
    if len(tslots) > 1:
        errors.append(f"[{ep_name}] multiple T_slots values across scenarios: {sorted(tslots)}")
    return errors

# ---------- Orchestrator over ALL datasets (episode-first) and ALL topologies ----------
def validate_everything_episode_first(datasets: dict, topologies: dict) -> dict:
    """
    'datasets' shape:
        {
          "ep_000": {
             "light":   {"episodes": df, "agents": df, "arrivals": df, "tasks": df},
             "moderate":{...},
             "heavy":   {...}
          },
          "ep_001": {...}
        }
    """
    report = {"datasets": {}, "episodes_consistency": {}, "topologies": {}, "pairs": {}}

    # 1) Validate each (episode, scenario)
    for ep_name, ep_pack in datasets.items():
        report["datasets"][ep_name] = {}
        for scenario, dpack in ep_pack.items():
            key = f"{ep_name}/{scenario}"
            errs = validate_one_dataset(key, dpack)
            report["datasets"][ep_name][scenario] = {"ok": len(errs) == 0, "errors": errs}

    # 2) Episode-level Delta/T_slots consistency across scenarios
    for ep_name, ep_pack in datasets.items():
        errs = validate_episode_delta_consistency(ep_name, ep_pack)
        report["episodes_consistency"][ep_name] = {"ok": len(errs) == 0, "errors": errs}

    # 3) Validate each topology
    for tname, tpack in topologies.items():
        errs = validate_one_topology(tname, tpack)
        report["topologies"][tname] = {"ok": len(errs) == 0, "errors": errs}

    # 4) Pairwise validation for every valid (ep, scenario) × valid topology
    for ep_name, ep_pack in datasets.items():
        for scenario, dpack in ep_pack.items():
            d_ok = report["datasets"][ep_name][scenario]["ok"]
            ep_ok = report["episodes_consistency"][ep_name]["ok"]
            for tname, tres in report["topologies"].items():
                key = f"{ep_name}/{scenario}__{tname}"
                if d_ok and ep_ok and tres["ok"]:
                    errs = validate_dataset_topology_pair(ep_name, scenario, dpack, tname, topologies[tname])
                    report["pairs"][key] = {"ok": len(errs) == 0, "errors": errs}
                else:
                    report["pairs"][key] = {"ok": False, "errors": ["Skipped due to upstream invalid dataset/episode/topology."]}

    return report

# ---------- Pretty printer ----------
def print_validation_report_episode_first(report: dict):
    print("=== DATASETS (episode/scenario) ===")
    for ep_name, ep_res in report["datasets"].items():
        for scenario, info in ep_res.items():
            status = "OK" if info["ok"] else "FAIL"
            print(f"[{status}] {ep_name}/{scenario}")
            for e in info["errors"]:
                print(f"  - {e}")

    print("\n=== EPISODE-LEVEL CONSISTENCY (Delta & T_slots) ===")
    for ep_name, info in report["episodes_consistency"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {ep_name}")
        for e in info["errors"]:
            print(f"  - {e}")

    print("\n=== TOPOLOGIES ===")
    for name, info in report["topologies"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {name}")
        for e in info["errors"]:
            print(f"  - {e}")

    print("\n=== (EPISODE/SCENARIO) × TOPOLOGY PAIRS ===")
    for key, info in report["pairs"].items():
        status = "OK" if info["ok"] else "FAIL"
        print(f"[{status}] {key}")
        for e in info["errors"]:
            print(f"  - {e}")

In [8]:
# ---- run the new validator ----
report = validate_everything_episode_first(datasets, topologies)
print_validation_report_episode_first(report)

all_ok = (
    all(info["ok"] for ep in report["datasets"].values() for info in ep.values())
    and all(info["ok"] for info in report["episodes_consistency"].values())
    and all(info["ok"] for info in report["topologies"].values())
    and all(info["ok"] for info in report["pairs"].values())
)
if not all_ok:
    raise RuntimeError("Validation failed. See printed report for details.")

=== DATASETS (episode/scenario) ===
[OK] ep_000/heavy
[OK] ep_000/light
[OK] ep_000/moderate

=== EPISODE-LEVEL CONSISTENCY (Delta & T_slots) ===
[OK] ep_000

=== TOPOLOGIES ===
[OK] clustered
[OK] full_mesh
[OK] sparse_ring

=== (EPISODE/SCENARIO) × TOPOLOGY PAIRS ===
[OK] ep_000/heavy__clustered
[OK] ep_000/heavy__full_mesh
[OK] ep_000/heavy__sparse_ring
[OK] ep_000/light__clustered
[OK] ep_000/light__full_mesh
[OK] ep_000/light__sparse_ring
[OK] ep_000/moderate__clustered
[OK] ep_000/moderate__full_mesh
[OK] ep_000/moderate__sparse_ring


<h3> 1.3. Units Alignment </h3>

In this section, we align units for all dataset episodes and scenarios
and run consistency checks against all topologies.
- Datasets: use Delta from episodes.csv; add per-slot helpers:
    agents.f_local_slot (cycles/slot), tasks.deadline_slots (integer or NaN)
    
- Topologies: capacities are already per-slot (generator multiplied by Δ);
    we only verify time_step == Delta and non-negative capacities.


In [9]:
# ===== Helpers: safe getters =====
def _get_delta(episodes_df: pd.DataFrame) -> float:
    # Expect a single Delta value in episodes; take the first row
    if "Delta" not in episodes_df.columns:
        raise ValueError("episodes.csv must contain a 'Delta' column.")
    return float(episodes_df["Delta"].iloc[0])

def _ensure_numeric_positive(name: str, arr: np.ndarray):
    # Basic sanity: finite and no negatives for capacities/links
    if not np.isfinite(arr).all():
        raise ValueError(f"{name} contains non-finite values.")
    if (arr < 0).any():
        raise ValueError(f"{name} contains negative values.")

# ===== Alignment: per-dataset (one (episode, scenario) pack) =====
def align_units_for_dataset(dataset: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Given one dataset dict {"episodes","agents","arrivals","tasks"},
    return a copy with aligned/derived columns (per-slot helpers).
    """
    episodes = dataset["episodes"].copy()
    agents   = dataset["agents"].copy()
    arrivals = dataset["arrivals"].copy()
    tasks    = dataset["tasks"].copy()

    Delta = _get_delta(episodes)

    # Agents: add per-slot compute capacity helper (cycles/slot)
    if "f_local" not in agents.columns:
        raise ValueError("agents.csv must contain 'f_local'.")
    agents["f_local"] = agents["f_local"].astype(float)
    agents["f_local_slot"] = agents["f_local"] * Delta

    # Memory is MB; keep as float
    if "m_local" in agents.columns:
        agents["m_local"] = agents["m_local"].astype(float)

    # Tasks: ensure integer arrival slot
    if "t_arrival_slot" not in tasks.columns:
        raise ValueError("tasks.csv must contain 't_arrival_slot'.")
    tasks["t_arrival_slot"] = tasks["t_arrival_slot"].astype(int)

    # Build deadline_slots = ceil(deadline_s / Delta) when has_deadline == 1, else NaN
    if "has_deadline" in tasks.columns and "deadline_s" in tasks.columns:
        def _to_deadline_slots(row):
            if int(row["has_deadline"]) == 1 and np.isfinite(row["deadline_s"]):
                return int(math.ceil(float(row["deadline_s"]) / Delta))
            return np.nan
        tasks["deadline_slots"] = tasks.apply(_to_deadline_slots, axis=1)
        # Keep as nullable integer when possible
        try:
            tasks["deadline_slots"] = tasks["deadline_slots"].astype("Int64")
        except Exception:
            pass

    # Ensure key numeric task fields are floats
    for col in ["b_mb", "rho_cyc_per_mb", "c_cycles", "mem_mb"]:
        if col in tasks.columns:
            tasks[col] = tasks[col].astype(float)

    return {
        "episodes": episodes,
        "agents":   agents,
        "arrivals": arrivals,
        "tasks":    tasks,
    }

# ===== Verification: per-topology against a target Delta =====
def verify_topology_units(topology: Dict[str, Any], target_Delta: float) -> Tuple[bool, str]:
    """
    Ensure topology capacities are per-slot and consistent with dataset Delta:
    - time_step == target_Delta
    - shapes are valid (K x (K+1))
    - capacities non-negative
    Returns (ok, message).
    """
    # time_step check
    ts = float(topology.get("time_step", -1.0))
    if not np.isclose(ts, target_Delta, atol=1e-9):
        return (False, f"time_step mismatch (topology={ts}, dataset Delta={target_Delta})")

    # K and lists
    K = int(topology.get("number_of_servers", -1))
    priv = np.array(topology.get("private_cpu_capacities", []), dtype=float)
    pub  = np.array(topology.get("public_cpu_capacities", []), dtype=float)
    cloud = float(topology.get("cloud_computational_capacity", -1.0))
    M = np.array(topology.get("connection_matrix", []), dtype=float)

    if K <= 0:
        return (False, "Invalid 'number_of_servers' (K<=0).")
    if priv.shape[0] != K or pub.shape[0] != K:
        return (False, "private/public capacities must have length K.")
    if M.shape != (K, K+1):
        return (False, f"connection_matrix shape must be (K, K+1), got {M.shape}.")

    # Non-negative checks
    _ensure_numeric_positive("private_cpu_capacities", priv)
    _ensure_numeric_positive("public_cpu_capacities",  pub)
    if not np.isfinite(cloud) or cloud < 0:
        return (False, "cloud_computational_capacity must be non-negative and finite.")
    _ensure_numeric_positive("connection_matrix", M)

    return (True, "topology verified (per-slot, consistent).")

# ===== Batch alignment for ALL datasets (episode-first) & ALL topologies =====
def align_all_units_episode_first(
    datasets_ep_first: Dict[str, Dict[str, Dict[str, pd.DataFrame]]],
    topologies_by_name: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Input 'datasets_ep_first' shape:
        {
          "ep_000": {
             "light":   {"episodes": df, "agents": df, "arrivals": df, "tasks": df},
             "moderate":{...},
             "heavy":   {...}
          },
          "ep_001": {...}
        }

    Returns:
        {
          "datasets_aligned": { ep_name: { scenario: aligned_pack, ... }, ... },
          "topology_checks":  { topo_name: { ep_name: { scenario: {ok, message} } } }
        }
    """
    out = {
        "datasets_aligned": {},
        "topology_checks":  {}
    }

    # Align datasets (episode/scenario)
    for ep_name, ep_pack in datasets_ep_first.items():
        out["datasets_aligned"][ep_name] = {}
        for scenario, ds in ep_pack.items():
            try:
                out["datasets_aligned"][ep_name][scenario] = align_units_for_dataset(ds)
            except Exception as e:
                raise RuntimeError(f"[{ep_name}/{scenario}] dataset alignment failed: {e}") from e

    # Verify each topology against each (episode, scenario) Delta
    for topo_name, topo_bundle in topologies_by_name.items():
        topo_obj = topo_bundle.get("topology_data", None)
        if not isinstance(topo_obj, dict):
            raise RuntimeError(f"[{topo_name}] 'topology_data' missing or not a dict.")
        out["topology_checks"][topo_name] = {}

        for ep_name, ep_pack in out["datasets_aligned"].items():
            out["topology_checks"][topo_name][ep_name] = {}
            for scenario, aligned in ep_pack.items():
                Delta = _get_delta(aligned["episodes"])
                ok, msg = verify_topology_units(topo_obj, Delta)
                out["topology_checks"][topo_name][ep_name][scenario] = {"ok": bool(ok), "message": msg}

    return out

# ===== Pretty printer (episode-first) =====
def print_alignment_summary_episode_first(result: Dict[str, Any]):
    # Datasets
    print("=== DATASETS (aligned, episode/scenario) ===")
    for ep_name in sorted(result["datasets_aligned"].keys()):
        for scenario in sorted(result["datasets_aligned"][ep_name].keys()):
            ds = result["datasets_aligned"][ep_name][scenario]
            Delta = _get_delta(ds["episodes"])
            n_tasks = len(ds["tasks"])
            n_agents = len(ds["agents"])
            print(f"[{ep_name}/{scenario}] Delta={Delta}  tasks={n_tasks}  agents={n_agents}")

    # Topologies
    print("\n=== TOPOLOGIES (checks vs each episode/scenario) ===")
    for topo_name, by_ep in result["topology_checks"].items():
        print(f"Topology: {topo_name}")
        for ep_name in sorted(by_ep.keys()):
            for scenario in sorted(by_ep[ep_name].keys()):
                r = by_ep[ep_name][scenario]
                flag = "OK" if r["ok"] else "FAIL"
                print(f"  - {ep_name}/{scenario}: {flag}  -> {r['message']}")

In [10]:
# ==== Example driver (after your loading step) ====
# datasets: episode-first dict
# topologies: { "full_mesh": {...}, "clustered": {...}, "sparse_ring": {...} }

result_align = align_all_units_episode_first(datasets_ep_first=datasets, topologies_by_name=topologies)
print_alignment_summary_episode_first(result_align)

# Access aligned data for a specific episode/scenario:
print("\n ===EXAMPLE===")
aligned_light_ep0 = result_align["datasets_aligned"]["ep_000"]["light"]
agents_ep0_light  = aligned_light_ep0["agents"]   # has f_local_slot
tasks_ep0_light   = aligned_light_ep0["tasks"]    # has deadline_slots

=== DATASETS (aligned, episode/scenario) ===
[ep_000/heavy] Delta=1.0  tasks=30636  agents=18
[ep_000/light] Delta=1.0  tasks=2113  agents=18
[ep_000/moderate] Delta=1.0  tasks=8262  agents=18

=== TOPOLOGIES (checks vs each episode/scenario) ===
Topology: clustered
  - ep_000/heavy: OK  -> topology verified (per-slot, consistent).
  - ep_000/light: OK  -> topology verified (per-slot, consistent).
  - ep_000/moderate: OK  -> topology verified (per-slot, consistent).
Topology: full_mesh
  - ep_000/heavy: OK  -> topology verified (per-slot, consistent).
  - ep_000/light: OK  -> topology verified (per-slot, consistent).
  - ep_000/moderate: OK  -> topology verified (per-slot, consistent).
Topology: sparse_ring
  - ep_000/heavy: OK  -> topology verified (per-slot, consistent).
  - ep_000/light: OK  -> topology verified (per-slot, consistent).
  - ep_000/moderate: OK  -> topology verified (per-slot, consistent).


<h3> 1.4. Build Scenario–Topology Pairs </h3>

In this step, all datasets are paired with all topologies (Cartesian product). Each pair is checked for matching time parameters, then a basic bundle is created for further enrichment.

In [11]:
def _delta_from_episodes(episodes_df: pd.DataFrame) -> float:
    """Extract a single Delta value from episodes table."""
    if "Delta" not in episodes_df.columns:
        raise ValueError("episodes.csv must contain 'Delta'.")
    return float(episodes_df["Delta"].iloc[0])

def _topology_time_step(topo_json: Dict[str, Any]) -> float:
    """Extract the topology time_step."""
    ts = topo_json.get("time_step", None)
    if ts is None:
        raise ValueError("topology.json must contain 'time_step'.")
    return float(ts)

def build_topology_episode_pairs(
    datasets_ep_first: Dict[str, Dict[str, Dict[str, pd.DataFrame]]],
    topologies: Dict[str, Dict[str, Any]],
    strict_delta_match: bool = True
) -> Dict[str, Dict[str, Dict[str, Any]]]:
    """
    Build pairs between every topology and every (episode, scenario) dataset.
    If strict_delta_match is True, any mismatch between dataset Delta and topology time_step raises an error.
    """
    pairs_by_topology: Dict[str, Dict[str, Dict[str, Any]]] = {}

    # Iterate topologies first (topology-centric)
    for topo_name, topo_bundle in topologies.items():
        topo_data = topo_bundle.get("topology_data", None)
        meta_data = topo_bundle.get("meta_data", None)
        cm_df     = topo_bundle.get("connection_matrix", None)

        if not isinstance(topo_data, dict):
            raise ValueError(f"[{topo_name}] topology_data missing or not a dict.")
        if cm_df is None:
            raise ValueError(f"[{topo_name}] connection_matrix DataFrame is missing.")

        # Validate K and connection matrix shape
        K = int(topo_data.get("number_of_servers", -1))
        if K <= 0:
            raise ValueError(f"[{topo_name}] invalid 'number_of_servers' in topology.json")
        if not (cm_df.shape[0] == K and cm_df.shape[1] == K + 1):
            raise ValueError(
                f"[{topo_name}] connection_matrix shape must be (K, K+1); got {cm_df.shape}"
            )

        topo_ts = _topology_time_step(topo_data)

        # Prepare container for this topology
        if topo_name not in pairs_by_topology:
            pairs_by_topology[topo_name] = {}

        # Compare with every (episode, scenario)
        for ep_name, scenarios in datasets_ep_first.items():
            if ep_name not in pairs_by_topology[topo_name]:
                pairs_by_topology[topo_name][ep_name] = {}

            for scen_name, ds in scenarios.items():
                ds_Delta = _delta_from_episodes(ds["episodes"])
                delta_ok = bool(np.isclose(ds_Delta, topo_ts, atol=1e-12))
                msg = "OK" if delta_ok else (
                    f"time_step mismatch (dataset Delta={ds_Delta}, topology time_step={topo_ts})"
                )
                if (not delta_ok) and strict_delta_match:
                    raise ValueError(f"[{topo_name} × {ep_name}/{scen_name}] {msg}")

                # Store bundle
                pairs_by_topology[topo_name][ep_name][scen_name] = {
                    "scenario": scen_name,
                    "episode": ep_name,
                    "topology": topo_name,
                    "Delta": ds_Delta,
                    "K": K,
                    "dataset": ds,
                    "topology_data": topo_data,
                    "topology_meta_data": meta_data,
                    "connection_matrix_df": cm_df,
                    "checks": {"delta_match": delta_ok, "message": msg}
                }

    return pairs_by_topology

def print_pairs_summary_topology_first_ep(
    pairs_by_topology: Dict[str, Dict[str, Dict[str, Any]]]
) -> None:
    """Pretty-print summary as topology → episode → scenario."""
    print("=== TOPOLOGY × EPISODE × SCENARIO ===")
    for topo_name, by_ep in pairs_by_topology.items():
        print(f"[TOPOLOGY] {topo_name}")
        for ep_name in sorted(by_ep.keys()):
            print(f"  ├─ Episode: {ep_name}")
            scen_map = by_ep[ep_name]
            for scen_name in sorted(scen_map.keys()):
                bundle = scen_map[scen_name]
                flag  = "OK" if bundle["checks"]["delta_match"] else "FAIL"
                K     = bundle["K"]
                Delta = bundle["Delta"]
                msg   = bundle["checks"]["message"]
                print(f"  │    - [{flag}] {scen_name:9s} | K={K:2d}  Δ={Delta:g}  -> {msg}")

In [12]:
# --- Example driver (with your current variables) ---
pairs_by_topology = build_topology_episode_pairs(
    datasets_ep_first=datasets,   # episode-first dict you already built
    topologies=topologies,
    strict_delta_match=True
)

print_pairs_summary_topology_first_ep(pairs_by_topology)

# Access examples:
print("\n ===EXAMPLE===")
pairs_by_topology["full_mesh"]["ep_000"]["light"]["dataset"]["tasks"]
pairs_by_topology["clustered"]["ep_000"]["heavy"]["connection_matrix_df"]

=== TOPOLOGY × EPISODE × SCENARIO ===
[TOPOLOGY] clustered
  ├─ Episode: ep_000
  │    - [OK] heavy     | K=18  Δ=1  -> OK
  │    - [OK] light     | K=18  Δ=1  -> OK
  │    - [OK] moderate  | K=18  Δ=1  -> OK
[TOPOLOGY] full_mesh
  ├─ Episode: ep_000
  │    - [OK] heavy     | K=18  Δ=1  -> OK
  │    - [OK] light     | K=18  Δ=1  -> OK
  │    - [OK] moderate  | K=18  Δ=1  -> OK
[TOPOLOGY] sparse_ring
  ├─ Episode: ep_000
  │    - [OK] heavy     | K=18  Δ=1  -> OK
  │    - [OK] light     | K=18  Δ=1  -> OK
  │    - [OK] moderate  | K=18  Δ=1  -> OK


<h3> 1.5. Agent→MEC mapping (for all pairs) </h3>

Agent → MEC Mapping assigns each agent to a specific MEC server.
This creates a fixed mec_id for every agent (e.g., agent_id % K), which determines where its tasks are initially queued and processed in the MDP environment.

In [13]:
def assign_agents_to_mecs(pairs_by_topology):
    """
    Adds agent→MEC mapping to each (topology / ep / scenario) bundle.
    - Rule: mec_id = agent_id % K
    - Writes:
        bundle["agent_to_mec"]                  (pd.Series, index=agent_id)
        bundle["dataset"]["agents"]["mec_id"]   (added column)
    """
    for topo_name, by_ep in pairs_by_topology.items():
        for ep_name, by_scen in by_ep.items():
            for scen_name, bundle in by_scen.items():

                ds = bundle["dataset"]
                agents = ds["agents"].copy()
                K = int(bundle["K"])

                if "agent_id" not in agents.columns:
                    raise ValueError(f"[{topo_name}/{ep_name}/{scen_name}] agents.csv missing 'agent_id'.")

                # Ensure agent_id is contiguous & sorted (0..N_agents-1)
                agents = agents.sort_values("agent_id").reset_index(drop=True)
                expected_n = int(bundle["dataset"]["episodes"]["N_agents"].iloc[0])
                if agents["agent_id"].min() != 0 or agents["agent_id"].max() != expected_n - 1:
                    raise ValueError(f"[{topo_name}/{ep_name}/{scen_name}] agent_id range not 0..N_agents-1.")

                # Mapping
                mec_ids = (agents["agent_id"].astype(int) % K).astype(int)
                agents["mec_id"] = mec_ids

                # Store: dataset copy + Series with index=agent_id
                ds["agents"] = agents
                bundle["agent_to_mec"] = pd.Series(
                    data=mec_ids.values,
                    index=agents["agent_id"].values,
                    name="mec_id"
                )

    return pairs_by_topology

In [14]:
# Apply mapping
pairs_by_topology = assign_agents_to_mecs(pairs_by_topology)

# Quick sanity peek
print("\n ===EXAMPLE===")
pairs_by_topology["clustered"]["ep_000"]["heavy"]["dataset"]["agents"].head()

,agent_id,f_local,m_local,lam_sec,mec_id
0,0,1.741183e+09,5713.849721,0.708673,0
1,1,1.352326e+09,4566.428755,0.234989,1
2,2,1.726668e+09,5815.120004,0.228174,2
3,3,1.543616e+09,3539.850245,0.310369,3
4,4,1.130883e+09,4161.367769,0.548990,4


<h3> 1.6. Environment Configuration </h3>

In this step, we build a unified env_config for each scenario–topology pair.
It bundles all required information for the MDP/RL environment—such as compute capacities, the Agent→MEC mapping, connection matrix, initial queue states, and action/state specifications—into a single consistent configuration used by the RL training process.

In [15]:
def _extract_core_from_bundle(bundle: Dict[str, Any]) -> Dict[str, Any]:
    required = ["dataset", "topology_data", "connection_matrix_df", "Delta", "K"]
    for k in required:
        if k not in bundle:
            raise ValueError(f"Bundle missing required key: '{k}'")

    ds   = bundle["dataset"]
    topo = bundle["topology_data"]
    Mdf  = bundle["connection_matrix_df"]

    private_cpu = np.asarray(topo["private_cpu_capacities"], dtype=float)
    public_cpu  = np.asarray(topo["public_cpu_capacities"],  dtype=float)
    cloud_cpu   = float(topo["cloud_computational_capacity"])
    M           = Mdf.to_numpy(dtype=float)  # (K, K+1), last col MEC→Cloud (MB/slot)

    return dict(
        Delta=float(bundle["Delta"]),
        K=int(bundle["K"]),
        episodes=ds["episodes"],
        agents=ds["agents"],
        arrivals=ds["arrivals"],
        tasks=ds["tasks"],
        private_cpu=private_cpu,
        public_cpu=public_cpu,
        cloud_cpu=cloud_cpu,
        connection_matrix=M,
        topology_type=topo.get("topology_type", "unknown"),
    )

def _build_default_queues(K: int) -> Dict[str, np.ndarray]:
    return {
        "mec_local_cycles":   np.zeros(K, dtype=float),
        "mec_public_cycles":  np.zeros(K, dtype=float),
        "mec_bytes_in_transit": np.zeros(K, dtype=float),
        "cloud_cycles":       np.array([0.0], dtype=float),
    }

def _derive_action_space() -> Dict[str, Any]:
    return {"type": "discrete", "n": 3, "labels": {0: "LOCAL", 1: "MEC", 2: "CLOUD"}}

def _derive_state_spec(K: int) -> Dict[str, Any]:
    return {
        "components": {
            "queues": {
                "mec_local_cycles":  {"shape": (K,),   "dtype": "float"},
                "mec_public_cycles": {"shape": (K,),   "dtype": "float"},
                "cloud_cycles":      {"shape": (1,),   "dtype": "float"},
            },
            "links": {
                "connection_matrix": {"shape": (K, K+1), "dtype": "float"},
            },
            "capacities": {
                "private_cpu": {"shape": (K,), "dtype": "float"},
                "public_cpu":  {"shape": (K,), "dtype": "float"},
                "cloud_cpu":   {"shape": (1,), "dtype": "float"},
            }
        },
        "note": "Declarative spec; tensor assembly happens in the Env at each step."
    }

def build_env_config_for_bundle(bundle: Dict[str, Any]) -> Dict[str, Any]:
    core = _extract_core_from_bundle(bundle)

    if "agent_to_mec" not in bundle:
        raise ValueError("Bundle has no 'agent_to_mec' mapping. Run Stage 5 first.")

    agent_to_mec = bundle["agent_to_mec"]
    if isinstance(agent_to_mec, pd.Series):
        # reorder by agent_id if needed
        if agent_to_mec.index.name != "agent_id":
            agent_to_mec.index.name = "agent_id"
        idx = core["agents"].sort_values("agent_id")["agent_id"].to_numpy()
        agent_to_mec = agent_to_mec.reindex(idx)
        agent_to_mec_arr = agent_to_mec.to_numpy(dtype=int)
    else:
        agent_to_mec_arr = np.asarray(agent_to_mec, dtype=int)

    N_agents = int(core["episodes"]["N_agents"].iloc[0])
    if len(agent_to_mec_arr) != N_agents:
        raise ValueError(f"agent_to_mec length ({len(agent_to_mec_arr)}) != N_agents ({N_agents}).")

    queues_init  = _build_default_queues(core["K"])
    action_space = _derive_action_space()
    state_spec   = _derive_state_spec(core["K"])

    env_config = {
        "Delta": core["Delta"],
        "K": core["K"],
        "topology_type": core["topology_type"],
        "connection_matrix": core["connection_matrix"],

        "private_cpu": core["private_cpu"],
        "public_cpu":  core["public_cpu"],
        "cloud_cpu":   core["cloud_cpu"],

        "N_agents": N_agents,
        "agent_to_mec": agent_to_mec_arr,

        # aligned dataframes
        "episodes": core["episodes"],
        "agents":   core["agents"],
        "arrivals": core["arrivals"],
        "tasks":    core["tasks"],

        "queues_initial": queues_init,
        "action_space": action_space,
        "state_spec": state_spec,

        "checks": bundle.get("checks", {"delta_match": True, "message": "n/a"}),
    }
    return env_config

def build_all_env_configs(
    pairs_by_topology: Dict[str, Dict[str, Dict[str, Any]]]
) -> Dict[str, Dict[str, Dict[str, Dict[str, Any]]]]:
    """
    Build env_config for every (topology / episode / scenario) bundle.

    Desired output shape (EPISODE-first):
        env_configs[episode][topology][scenario] = env_config

    So you can access:
        env_configs["ep_000"]["clustered"]["heavy"]["agent_to_mec"]
    """
    out: Dict[str, Dict[str, Dict[str, Dict[str, Any]]]] = {}
    # pairs_by_topology: topo -> ep -> scen -> bundle
    for topo_name, by_ep in pairs_by_topology.items():
        for ep_name, by_scen in by_ep.items():
            # ensure episode level exists
            if ep_name not in out:
                out[ep_name] = {}
            # ensure topology level under this episode exists
            if topo_name not in out[ep_name]:
                out[ep_name][topo_name] = {}
            for scen_name, bundle in by_scen.items():
                if "agent_to_mec" not in bundle:
                    raise RuntimeError(
                        f"[{topo_name}/{ep_name}/{scen_name}] missing 'agent_to_mec'. Run Stage 5 first."
                    )
                env_cfg = build_env_config_for_bundle(bundle)
                out[ep_name][topo_name][scen_name] = env_cfg
    return out

In [16]:
# Build
env_configs = build_all_env_configs(pairs_by_topology)

# Example access:
print("\n ===EXAMPLE===")
env_configs["ep_000"]["clustered"]["heavy"]["agent_to_mec"]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

<h3> 1.7. Sanity Checks </h3>

In this step, we verify that each env_config is internally consistent (queue shapes, capacities, agent→MEC mapping, and connection matrix are valid and ready for simulation).

In [17]:
def sanity_check_env_config(env_config):
    errors = []

    # 1) Agent → MEC alignment
    N_agents = env_config["N_agents"]
    if len(env_config["agent_to_mec"]) != N_agents:
        errors.append("Length of agent_to_mec does not match N_agents.")

    # 2) Queue initial state shapes
    K = env_config["K"]
    q = env_config["queues_initial"]
    if q["mec_local_cycles"].shape != (K,):
        errors.append("mec_local_cycles queue shape mismatch.")
    if q["mec_public_cycles"].shape != (K,):
        errors.append("mec_public_cycles queue shape mismatch.")
    if q["mec_bytes_in_transit"].shape != (K,):
        errors.append("mec_bytes_in_transit queue shape mismatch.")
    if q["cloud_cycles"].shape != (1,):
        errors.append("cloud_cycles shape mismatch (should be (1,)).")

    # 3) Non-negative compute capacities
    if (env_config["private_cpu"] < 0).any():
        errors.append("private_cpu has negative values.")
    if (env_config["public_cpu"] < 0).any():
        errors.append("public_cpu has negative values.")
    if env_config["cloud_cpu"] < 0:
        errors.append("cloud_cpu is negative.")

    # 4) Connection matrix dimension (K x K+1)
    M = env_config["connection_matrix"]
    if M.shape != (K, K+1):
        errors.append("connection_matrix shape mismatch.")

    # 5) Action space correctness
    if env_config["action_space"]["type"] != "discrete":
        errors.append("Action space must be discrete (LOCAL/MEC/CLOUD).")

    return errors

def sanity_check_all(env_configs):
    for topo_name, by_ep in env_configs.items():
        for ep_name, by_scen in by_ep.items():
            for scen_name, env_cfg in by_scen.items():
                errs = sanity_check_env_config(env_cfg)
                if errs:
                    print(f"[FAIL] {topo_name}/{ep_name}/{scen_name}:")
                    for e in errs:
                        print("   -", e)
                else:
                    print(f"[OK]   {topo_name}/{ep_name}/{scen_name}")

[OK]   ep_000/clustered/heavy
[OK]   ep_000/clustered/light
[OK]   ep_000/clustered/moderate
[OK]   ep_000/full_mesh/heavy
[OK]   ep_000/full_mesh/light
[OK]   ep_000/full_mesh/moderate
[OK]   ep_000/sparse_ring/heavy
[OK]   ep_000/sparse_ring/light
[OK]   ep_000/sparse_ring/moderate


In [18]:
# Run all sanity checks
sanity_check_all(env_configs)

print("env_configs: \n", env_configs)

print("\n ===EXAMPLE===")
display(env_configs["ep_000"]["clustered"]["heavy"]["tasks"])

{'ep_000': {'clustered': {'heavy': {'Delta': 1.0,
    'K': 18,
    'topology_type': 'clustered',
    'connection_matrix': array([[  0.        ,   9.79880061,  11.95829539,  11.47040438,
              9.98019544,   8.81405007,   0.        ,   0.        ,
              0.        ,   0.        ,   0.        ,   0.        ,
              0.        ,   0.        ,   0.        ,   0.        ,
              0.        ,   0.        ,  98.91279982],
           [  9.79880061,   0.        ,   8.37514324,  11.53560011,
             10.70258438,   9.13689312,   0.        ,   0.        ,
              0.        ,   0.        ,   0.        ,   0.        ,
              0.        ,   0.        ,   0.        ,   0.        ,
              0.        ,   0.        , 104.60920208],
           [ 11.95829539,   8.37514324,   0.        ,   8.81463191,
              9.63771407,   9.17070042,   0.        ,   0.        ,
              0.        ,   0.        ,   0.        ,   0.        ,
              0.        

Saving the Information

In [20]:
def _summarize_array(arr, max_items=6):
    """Return a short, readable summary string for numpy arrays."""
    try:
        arr = np.asarray(arr)
        base = f"ndarray shape={arr.shape}, dtype={arr.dtype}"
        if arr.size == 0:
            return base + " | empty"
        # Show a few values only if it's 1D or small
        if arr.ndim == 1 and arr.size <= max_items:
            return base + f" | values={arr.tolist()}"
        # Stats if numeric
        if np.issubdtype(arr.dtype, np.number):
            return base + f" | min={np.nanmin(arr):.4g}, max={np.nanmax(arr):.4g}, mean={np.nanmean(arr):.4g}"
        return base
    except Exception as e:
        return f"(array summary failed: {e})"

def _summarize_df(df: pd.DataFrame, max_cols=10):
    """Return a short summary string for DataFrames."""
    try:
        cols = df.columns.tolist()
        cols_show = cols[:max_cols] + (["..."] if len(cols) > max_cols else [])
        return f"DataFrame shape={df.shape}, columns={cols_show}"
    except Exception as e:
        return f"(dataframe summary failed: {e})"

def _summarize_any(name, obj, indent="    "):
    """Create a few readable lines summarizing an object by type."""
    lines = []
    if isinstance(obj, pd.DataFrame):
        lines.append(f"{indent}{name}: {_summarize_df(obj)}")
    elif isinstance(obj, np.ndarray):
        lines.append(f"{indent}{name}: {_summarize_array(obj)}")
    elif isinstance(obj, (list, tuple)):
        preview = obj[:6] if len(obj) > 6 else obj
        lines.append(f"{indent}{name}: list len={len(obj)}, preview={preview}")
    elif isinstance(obj, dict):
        lines.append(f"{indent}{name}: dict keys={list(obj.keys())}")
        # If it's the queues dict or small dict, briefly dive one level
        if name == "queues_initial" or len(obj) <= 6:
            for k, v in obj.items():
                sub = _summarize_any(k, v, indent=indent + "  ")
                lines.extend(sub if isinstance(sub, list) else [sub])
    elif isinstance(obj, (int, float, str, bool, type(None))):
        lines.append(f"{indent}{name}: {repr(obj)}")
    else:
        # Try numpy conversion
        try:
            arr = np.asarray(obj)
            lines.append(f"{indent}{name}: {_summarize_array(arr)}")
        except Exception:
            lines.append(f"{indent}{name}: ({type(obj).__name__})")
    return lines

def save_env_configs_text(env_configs, out_path="./artifacts/env_configs_summary.txt"):
    """
    Save a human-readable summary of env_configs (episode → topology → scenario) to a text file.
    Includes shapes of DataFrames, key columns, array shapes/stats, and key scalar parameters.
    """
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    lines = []
    lines.append("=== ENV CONFIGS SUMMARY (episode → topology → scenario) ===\n")

    # Iterate deterministic order for reproducibility
    for ep_name in sorted(env_configs.keys()):
        lines.append(f"[EPISODE] {ep_name}")
        by_topo = env_configs[ep_name]
        for topo_name in sorted(by_topo.keys()):
            lines.append(f"  [TOPOLOGY] {topo_name}")
            by_scen = by_topo[topo_name]
            for scen_name in sorted(by_scen.keys()):
                env_cfg = by_scen[scen_name]
                lines.append(f"    [SCENARIO] {scen_name}")

                # Highlight most relevant scalars first if present
                for key in ["Delta", "K", "N_agents", "topology_type"]:
                    if key in env_cfg:
                        lines.extend(_summarize_any(key, env_cfg[key], indent="      "))

                # Then summarize major tensors/arrays
                for key in ["connection_matrix", "private_cpu", "public_cpu", "cloud_cpu", "agent_to_mec"]:
                    if key in env_cfg:
                        lines.extend(_summarize_any(key, env_cfg[key], indent="      "))

                # Then summarize DataFrames (episodes, agents, arrivals, tasks)
                for key in ["episodes", "agents", "arrivals", "tasks"]:
                    if key in env_cfg:
                        lines.extend(_summarize_any(key, env_cfg[key], indent="      "))

                # Queues + RL descriptors (optional)
                for key in ["queues_initial", "action_space", "state_spec", "checks"]:
                    if key in env_cfg:
                        lines.extend(_summarize_any(key, env_cfg[key], indent="      "))

                lines.append("")  # blank line between scenarios

            lines.append("")  # blank line between topologies
        lines.append("")      # blank line between episodes

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print(f"[saved] env_configs summary → {out_path}")

# ---- usage ----
save_env_configs_text(env_configs, out_path="./artifacts/env_configs_summary.txt")

[saved] env_configs summary → ./artifacts/env_configs_summary.txt


At Step 1, we have loaded the data, aligned the units, assigned agents to MECs, and prepared the environment configuration. Finally, we have performed consistency checks to ensure the data is correct. Next, we can move on to task labeling.

<h1> Step 2: Task Labeling </h1>

<h3> 2.1. Basic Task Labeling (buckets, urgency, atomicity, ...) </h3>

In [21]:
# ---------- helpers: quantile-based cut points ----------
def _quantile_cutpoints(s: pd.Series, q_low=0.33, q_high=0.66) -> Tuple[float, float]:
    s = pd.to_numeric(s, errors="coerce").dropna()
    if len(s) == 0:
        return (np.nan, np.nan)
    return (float(s.quantile(q_low)), float(s.quantile(q_high)))

def _bucketize(value: float, q1: float, q2: float) -> str:
    # Returns 'S', 'M', 'L' based on two cut points (q1<=q2)
    if not np.isfinite(value) or not np.isfinite(q1) or not np.isfinite(q2):
        return "U"  # Unknown
    if value <= q1: return "S"
    if value <= q2: return "M"
    return "L"

# ---------- threshold builder (adaptive to each tasks DF) ----------
def build_task_label_thresholds(tasks_df: pd.DataFrame,
                                q_low=0.33, q_high=0.66,
                                urgent_slots_cap: int = 2) -> Dict[str, Any]:
    """
    Build adaptive thresholds from the data itself (per-episode/senario),
    so 'light/moderate/heavy' are handled robustly.
    """
    q_b_mb   = _quantile_cutpoints(tasks_df["b_mb"], q_low, q_high) if "b_mb" in tasks_df else (np.nan, np.nan)
    q_rho    = _quantile_cutpoints(tasks_df["rho_cyc_per_mb"], q_low, q_high) if "rho_cyc_per_mb" in tasks_df else (np.nan, np.nan)
    q_mem    = _quantile_cutpoints(tasks_df["mem_mb"], q_low, q_high) if "mem_mb" in tasks_df else (np.nan, np.nan)
    q_split  = _quantile_cutpoints(tasks_df.loc[tasks_df.get("non_atomic", 0)==1, "split_ratio"], q_low, q_high) \
               if "split_ratio" in tasks_df else (np.nan, np.nan)

    return {
        "b_mb":   {"q1": q_b_mb[0],  "q2": q_b_mb[1]},
        "rho":    {"q1": q_rho[0],   "q2": q_rho[1]},
        "mem":    {"q1": q_mem[0],   "q2": q_mem[1]},
        "split":  {"q1": q_split[0], "q2": q_split[1]},
        # If deadline_slots ≤ urgent_slots_cap → 'hard' (latency sensitive)
        "urgent_slots_cap": int(urgent_slots_cap),
    }

# ---------- main labeling for a single tasks DF ----------
def label_tasks_df(tasks_df: pd.DataFrame, Delta: float, thresholds: Dict[str, Any]) -> pd.DataFrame:
    """
    Add label columns to tasks_df (returns a COPY).
    Columns added:
      - size_bucket, compute_bucket, mem_bucket
      - deadline_slots (if missing), urgency (none/soft/hard)
      - atomicity, split_bucket
      - latency_sensitive, compute_heavy, io_heavy, memory_heavy (bools)
      - routing_hint (LOCAL/MEC/CLOUD)
    """
    df = tasks_df.copy()

    # --- ensure numeric types
    for col in ["b_mb", "rho_cyc_per_mb", "c_cycles", "mem_mb", "deadline_s", "split_ratio"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # --- deadline_slots (if not precomputed in Units Alignment)
    if "deadline_slots" not in df.columns:
        if "has_deadline" in df.columns and "deadline_s" in df.columns:
            df["deadline_slots"] = np.where(
                (df["has_deadline"] == 1) & np.isfinite(df["deadline_s"]),
                np.ceil(df["deadline_s"] / float(Delta)).astype("float"),
                np.nan
            )
        else:
            df["deadline_slots"] = np.nan

    # --- bucketize size/compute/memory
    b_q1, b_q2   = thresholds["b_mb"]["q1"], thresholds["b_mb"]["q2"]
    rho_q1, rho_q2 = thresholds["rho"]["q1"], thresholds["rho"]["q2"]
    mem_q1, mem_q2 = thresholds["mem"]["q1"], thresholds["mem"]["q2"]

    df["size_bucket"]    = df["b_mb"].apply(lambda x: _bucketize(x, b_q1, b_q2)) if "b_mb" in df else "U"
    df["compute_bucket"] = df["rho_cyc_per_mb"].apply(lambda x: _bucketize(x, rho_q1, rho_q2)) if "rho_cyc_per_mb" in df else "U"
    df["mem_bucket"]     = df["mem_mb"].apply(lambda x: _bucketize(x, mem_q1, mem_q2)) if "mem_mb" in df else "U"

    # --- atomicity & split buckets
    if "non_atomic" in df.columns:
        df["atomicity"] = np.where(df["non_atomic"] == 1, "splittable", "atomic")
    else:
        df["atomicity"] = "atomic"

    if "split_ratio" in df.columns:
        sp_q1, sp_q2 = thresholds["split"]["q1"], thresholds["split"]["q2"]
        df["split_bucket"] = np.where(
            df["atomicity"] == "splittable",
            df["split_ratio"].apply(lambda v: _bucketize(v, sp_q1, sp_q2)),
            "NA"
        )
    else:
        df["split_bucket"] = "NA"

    # --- urgency levels
    urgent_cap = int(thresholds.get("urgent_slots_cap", 2))
    def _urg(row):
        if int(row.get("has_deadline", 0)) != 1 or not np.isfinite(row.get("deadline_slots", np.nan)):
            return "none"
        slots = int(row["deadline_slots"])
        if slots <= urgent_cap:  # very tight deadline
            return "hard"
        return "soft"
    df["urgency"] = df.apply(_urg, axis=1)

    # --- boolean convenience labels
    df["latency_sensitive"] = (df["urgency"] == "hard")
    df["compute_heavy"]     = (df["compute_bucket"] == "L")
    df["io_heavy"]          = (df["size_bucket"] == "L")
    df["memory_heavy"]      = (df["mem_bucket"] == "L")

    # --- a very simple routing hint (only for debugging/EDA; not used by the RL policy)
    def _hint(row):
        if row["compute_heavy"] or row["memory_heavy"]:
            return "CLOUD"
        if row["latency_sensitive"]:
            return "MEC"
        return "LOCAL"
    df["routing_hint"] = df.apply(_hint, axis=1)

    return df

# ---------- batch apply to env_configs (topology → episode → scenario) ----------
def label_all_tasks_in_env_configs(env_configs: Dict[str, Dict[str, Dict[str, Any]]],
                                   q_low=0.33, q_high=0.66, urgent_slots_cap=2,
                                   verbose=True) -> Dict[str, Dict[str, Dict[str, Any]]]:
    """
    For each env_config:
      - build thresholds from its own tasks DF
      - label tasks
      - put labeled DF back into env_config["tasks"]
      - return a concise summary per bundle
    """
    summary = {}

    for topo_name, by_ep in env_configs.items():
        summary[topo_name] = {}
        for ep_name, by_scen in by_ep.items():
            summary[topo_name][ep_name] = {}
            for scen_name, env_cfg in by_scen.items():
                tasks = env_cfg["tasks"]
                Delta = float(env_cfg["Delta"])

                # thresholds adaptive to this bundle
                th = build_task_label_thresholds(tasks, q_low=q_low, q_high=q_high,
                                                 urgent_slots_cap=urgent_slots_cap)
                labeled = label_tasks_df(tasks, Delta=Delta, thresholds=th)
                env_cfg["tasks"] = labeled  # write back

                # tiny summary
                cnt = {
                    "n": len(labeled),
                    "urg_hard": int((labeled["urgency"] == "hard").sum()),
                    "splittable": int((labeled["atomicity"] == "splittable").sum()),
                    "size_L": int((labeled["size_bucket"] == "L").sum()),
                    "compute_L": int((labeled["compute_bucket"] == "L").sum()),
                    "mem_L": int((labeled["mem_bucket"] == "L").sum()),
                }
                summary[topo_name][ep_name][scen_name] = cnt

                if verbose:
                    print(f"[label] {topo_name}/{ep_name}/{scen_name} -> "
                          f"n={cnt['n']}, hard={cnt['urg_hard']}, split={cnt['splittable']}, "
                          f"sizeL={cnt['size_L']}, compL={cnt['compute_L']}, memL={cnt['mem_L']}")

    return env_configs, summary

In [22]:
# env_configs: Produced in Step 6 (structure: episode → topology → scenario)
env_configs, label_summary = label_all_tasks_in_env_configs(
    env_configs,
    q_low=0.33, q_high=0.66, urgent_slots_cap=2,  # tunable thresholds
    verbose=True
)

# Example access:
print("\n ===EXAMPLE===")
labeled_tasks = env_configs["ep_000"]["clustered"]["heavy"]["tasks"]
display(labeled_tasks.head())
print(labeled_tasks.info())

[label] ep_000/clustered/heavy -> n=30636, hard=10673, split=13766, sizeL=10416, compL=10416, memL=10416
[label] ep_000/clustered/light -> n=2113, hard=322, split=539, sizeL=719, compL=719, memL=719
[label] ep_000/clustered/moderate -> n=8262, hard=2117, split=2890, sizeL=2809, compL=2809, memL=2809
[label] ep_000/full_mesh/heavy -> n=30636, hard=10673, split=13766, sizeL=10416, compL=10416, memL=10416
[label] ep_000/full_mesh/light -> n=2113, hard=322, split=539, sizeL=719, compL=719, memL=719
[label] ep_000/full_mesh/moderate -> n=8262, hard=2117, split=2890, sizeL=2809, compL=2809, memL=2809
[label] ep_000/sparse_ring/heavy -> n=30636, hard=10673, split=13766, sizeL=10416, compL=10416, memL=10416
[label] ep_000/sparse_ring/light -> n=2113, hard=322, split=539, sizeL=719, compL=719, memL=719
[label] ep_000/sparse_ring/moderate -> n=8262, hard=2117, split=2890, sizeL=2809, compL=2809, memL=2809


,scenario,episode_id,task_id,agent_id,t_arrival_slot,t_arrival_time,b_mb,rho_cyc_per_mb,c_cycles,mem_mb,...,compute_bucket,mem_bucket,atomicity,split_bucket,urgency,latency_sensitive,compute_heavy,io_heavy,memory_heavy,routing_hint
0,heavy,0,0,0,0,0.0,7.202096,9.727147e+08,7.005585e+09,66.611010,...,S,M,atomic,NA,hard,True,False,True,False,MEC
1,heavy,0,1,1,0,0.0,5.479984,1.314973e+09,7.206031e+09,77.928800,...,M,L,atomic,NA,hard,True,False,False,True,CLOUD
2,heavy,0,2,4,0,0.0,8.421977,2.500222e+09,2.105681e+10,72.966446,...,L,M,splittable,S,hard,True,True,True,False,CLOUD
3,heavy,0,3,7,0,0.0,6.324986,1.779582e+09,1.125583e+10,56.492900,...,M,M,atomic,NA,hard,True,False,True,False,MEC
4,heavy,0,4,10,0,0.0,11.473269,1.087572e+09,1.247800e+10,73.389854,...,S,M,atomic,NA,hard,True,False,True,False,MEC


<h3> 2.2. Task Type Classification </h3>

In [24]:
# Pre-req: tasks already labeled by your previous step: 
#   size_bucket, compute_bucket, mem_bucket, urgency, atomicity, split_bucket, routing_hint, etc.

def _derive_task_type_row(row: pd.Series) -> tuple[str, str, str, list]:
    """
    Returns (task_type, task_subtype, type_reason, multi_flags)
      task_type   ∈ {"deadline_hard","latency_sensitive","compute_intensive","data_intensive","general"}
      task_subtype: finer note (e.g., "deadline_hard", "deadline_soft", ...)
      type_reason: short human-readable reason
      multi_flags: list of boolean tags that were true (for auditing)
    """

    # Collect boolean flags consistent with your earlier labeling:
    urgency        = str(row.get("urgency", "none"))         # "hard" | "soft" | "none"
    latency_flag   = (urgency == "hard") or (urgency == "soft")
    hard_deadline  = (urgency == "hard")

    compute_heavy  = bool(row.get("compute_heavy", False))   # compute_bucket == "L"
    memory_heavy   = bool(row.get("memory_heavy", False))    # mem_bucket == "L"
    io_heavy       = bool(row.get("io_heavy", False))        # size_bucket == "L"
    non_atomic     = bool(row.get("atomicity", "atomic") == "splittable")

    # Keep all active signals for audit:
    multi_flags = []
    if hard_deadline:  multi_flags.append("deadline_hard")
    elif latency_flag: multi_flags.append("deadline_soft")
    if compute_heavy:  multi_flags.append("compute_heavy")
    if memory_heavy:   multi_flags.append("memory_heavy")
    if io_heavy:       multi_flags.append("io_heavy")
    if non_atomic:     multi_flags.append("splittable")

    # --- Priority resolution (Chapter 4) ---
    # 1) Hard deadline dominates everything
    if hard_deadline:
        return ("deadline_hard", "deadline_hard", "hard deadline (tight slots)", multi_flags)

    # 2) Latency-sensitive (soft deadlines / delay-sensitive)
    if latency_flag:
        return ("latency_sensitive", "deadline_soft", "delay-sensitive (soft deadline)", multi_flags)

    # 3) Compute-intensive (c or rho or mem heavy)
    #    You may decide whether memory_heavy alone pushes to compute_intensive or creates a separate class.
    #    Based on Chapter 4 text we map memory_heavy into compute_intensive family.
    if compute_heavy or memory_heavy:
        return ("compute_intensive", "compute_or_memory_heavy", "high compute/memory demand", multi_flags)

    # 4) Data-intensive (mainly large input size / high IO pressure)
    if io_heavy:
        return ("data_intensive", "large_input_bandwidth", "large data volume / IO heavy", multi_flags)

    # 5) Otherwise general
    return ("general", "general", "no dominant constraint", multi_flags)

def apply_ch4_task_typing(tasks_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds Chapter-4 level task classes with priority rules into tasks_df (returns a COPY).
    Columns added:
      - task_type            (5-way class)
      - task_subtype         (finer descriptor)
      - type_reason          (short textual rationale)
      - multi_flags          (list of all active boolean traits)
    """
    df = tasks_df.copy()

    # Ensure the expected helper columns exist (created in your previous labeling step).
    required_cols = ["urgency", "compute_heavy", "memory_heavy", "io_heavy", "atomicity"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"apply_ch4_task_typing: missing label columns: {missing}")

    out_type, out_sub, out_reason, out_flags = [], [], [], []
    for _, r in df.iterrows():
        t, s, msg, flags = _derive_task_type_row(r)
        out_type.append(t)
        out_sub.append(s)
        out_reason.append(msg)
        out_flags.append(flags)

    df["task_type"]   = out_type
    df["task_subtype"]= out_sub
    df["type_reason"] = out_reason
    df["multi_flags"] = out_flags
    # For convenience: one-hot view (optional)
    df["is_general"]            = (df["task_type"] == "general")
    df["is_deadline_hard"]      = (df["task_type"] == "deadline_hard")
    df["is_latency_sensitive"]  = (df["task_type"] == "latency_sensitive")
    df["is_compute_intensive"]  = (df["task_type"] == "compute_intensive")
    df["is_data_intensive"]     = (df["task_type"] == "data_intensive")

    return df

def apply_task_typing_in_env_configs(env_configs: Dict[str, Dict[str, Dict[str, Any]]],
                                     verbose: bool = True) -> Dict[str, Dict[str, Dict[str, Any]]]:
    """
    env_configs structure (as we fixed earlier):
      env_configs[ep_name][topology_name][scenario_name]["tasks"] -> DataFrame

    This function:
      - applies Chapter-4 task typing to every tasks DF
      - writes back the enriched DataFrame
      - prints a short summary if verbose=True
    """
    for ep_name, by_topo in env_configs.items():
        for topo_name, by_scen in by_topo.items():
            for scen_name, env_cfg in by_scen.items():
                tasks = env_cfg["tasks"]
                enriched = apply_ch4_task_typing(tasks)
                env_cfg["tasks"] = enriched

                if verbose:
                    n = len(enriched)
                    counts = enriched["task_type"].value_counts().to_dict()
                    print(f"[typing] {ep_name}/{topo_name}/{scen_name}  n={n}  → {counts}")
    return env_configs

In [25]:
# ---- Run typing on your current env_configs (episode → topology → scenario) ----
env_configs = apply_task_typing_in_env_configs(env_configs, verbose=True)

# Example access:
print("\n ===EXAMPLE===")
env_configs["ep_000"]["clustered"]["heavy"]["tasks"][["task_id","task_type","task_subtype","type_reason","multi_flags"]].head()

[typing] ep_000/clustered/heavy  n=30636  → {'compute_intensive': 11370, 'deadline_hard': 10673, 'general': 5671, 'data_intensive': 2922}
[typing] ep_000/clustered/light  n=2113  → {'compute_intensive': 1009, 'general': 502, 'deadline_hard': 322, 'data_intensive': 280}
[typing] ep_000/clustered/moderate  n=8262  → {'compute_intensive': 3395, 'deadline_hard': 2117, 'general': 1829, 'data_intensive': 921}
[typing] ep_000/full_mesh/heavy  n=30636  → {'compute_intensive': 11370, 'deadline_hard': 10673, 'general': 5671, 'data_intensive': 2922}
[typing] ep_000/full_mesh/light  n=2113  → {'compute_intensive': 1009, 'general': 502, 'deadline_hard': 322, 'data_intensive': 280}
[typing] ep_000/full_mesh/moderate  n=8262  → {'compute_intensive': 3395, 'deadline_hard': 2117, 'general': 1829, 'data_intensive': 921}
[typing] ep_000/sparse_ring/heavy  n=30636  → {'compute_intensive': 11370, 'deadline_hard': 10673, 'general': 5671, 'data_intensive': 2922}
[typing] ep_000/sparse_ring/light  n=2113  → {

,task_id,task_type,task_subtype,type_reason,multi_flags
0,0,deadline_hard,deadline_hard,hard deadline (tight slots),"[deadline_hard, io_heavy]"
1,1,deadline_hard,deadline_hard,hard deadline (tight slots),"[deadline_hard, memory_heavy]"
2,2,deadline_hard,deadline_hard,hard deadline (tight slots),"[deadline_hard, compute_heavy, io_heavy, split..."
3,3,deadline_hard,deadline_hard,hard deadline (tight slots),"[deadline_hard, io_heavy]"
4,4,deadline_hard,deadline_hard,hard deadline (tight slots),"[deadline_hard, io_heavy]"


In [26]:
df = env_configs["ep_000"]["clustered"]["heavy"]["tasks"]
print(df["urgency"].value_counts())
print("\n", df["task_type"].value_counts())
print("\n", df.groupby("task_type")[["b_mb","rho_cyc_per_mb","mem_mb"]].median())

urgency
none    19963
hard    10673
Name: count, dtype: int64

 task_type
compute_intensive    11370
deadline_hard        10673
general               5671
data_intensive        2922
Name: count, dtype: int64

                        b_mb  rho_cyc_per_mb     mem_mb
task_type                                             
compute_intensive  4.946632    1.933898e+09  80.518150
data_intensive     8.263339    1.230034e+09  53.342833
deadline_hard      4.971393    1.496501e+09  63.870583
general            3.954205    1.223997e+09  53.340660


none → Tasks that we intend to do but do not have a specific deadline or time sensitivity </br>
hard → Tasks that have a very limited deadline and delay is very important to them

In [27]:
# Example access:
labeled_tasks_completed = env_configs["ep_000"]["clustered"]["heavy"]["tasks"]
labeled_tasks_completed.head()
labeled_tasks_completed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30636 entries, 0 to 30635
Data columns (total 38 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   scenario              30636 non-null  object 
 1   episode_id            30636 non-null  int64  
 2   task_id               30636 non-null  int64  
 3   agent_id              30636 non-null  int64  
 4   t_arrival_slot        30636 non-null  int64  
 5   t_arrival_time        30636 non-null  float64
 6   b_mb                  30636 non-null  float64
 7   rho_cyc_per_mb        30636 non-null  float64
 8   c_cycles              30636 non-null  float64
 9   mem_mb                30636 non-null  float64
 10  modality              30636 non-null  object 
 11  has_deadline          30636 non-null  int64  
 12  deadline_s            10673 non-null  float64
 13  deadline_time         10673 non-null  float64
 14  non_atomic            30636 non-null  int64  
 15  split_ratio        

Saving the Information

In [28]:
def _ensure_dir(path: str):
    """Create folder if it doesn't exist."""
    os.makedirs(path, exist_ok=True)

def save_all_env_configs(env_configs, out_root: str = "./artifacts/env_configs"):
    """
    Walk through env_configs (episode → topology → scenario) and save all data (tasks, agents, etc.) as CSV.
    Example output: ./artifacts/env_configs/ep_000/clustered/heavy/env_config.csv
    """
    n_saved = 0
    for ep_name, by_topo in env_configs.items():
        for topo_name, by_scen in by_topo.items():
            for scen_name, env_cfg in by_scen.items():
                out_dir = os.path.join(out_root, ep_name, topo_name, scen_name)
                _ensure_dir(out_dir)

                # Save tasks, agents, arrivals, and other components
                for df_name, df in env_cfg.items():
                    if isinstance(df, pd.DataFrame):
                        file_path_csv = os.path.join(out_dir, f"{df_name}_env_config.csv")
                        # file_path_pq = os.path.join(out_dir, f"{df_name}_env_config.parquet")

                        df.to_csv(file_path_csv, index=False)
                        # try:
                        #     df.to_parquet(file_path_pq, index=False)  # optional
                        # except Exception:
                        #     pass

                        print(f"[saved] {file_path_csv}  (rows={len(df)})")
                        n_saved += 1

    print(f"Done. Saved {n_saved} dataframes for env_configs (episode, topology, scenario).")

# Save ALL env_configs
save_all_env_configs(env_configs, out_root="./artifacts/env_configs")

[saved] ./artifacts/env_configs\ep_000\clustered\heavy\episodes_env_config.csv  (rows=1)
[saved] ./artifacts/env_configs\ep_000\clustered\heavy\agents_env_config.csv  (rows=18)
[saved] ./artifacts/env_configs\ep_000\clustered\heavy\arrivals_env_config.csv  (rows=30636)
[saved] ./artifacts/env_configs\ep_000\clustered\heavy\tasks_env_config.csv  (rows=30636)
[saved] ./artifacts/env_configs\ep_000\clustered\light\episodes_env_config.csv  (rows=1)
[saved] ./artifacts/env_configs\ep_000\clustered\light\agents_env_config.csv  (rows=18)
[saved] ./artifacts/env_configs\ep_000\clustered\light\arrivals_env_config.csv  (rows=2113)
[saved] ./artifacts/env_configs\ep_000\clustered\light\tasks_env_config.csv  (rows=2113)
[saved] ./artifacts/env_configs\ep_000\clustered\moderate\episodes_env_config.csv  (rows=1)
[saved] ./artifacts/env_configs\ep_000\clustered\moderate\agents_env_config.csv  (rows=18)
[saved] ./artifacts/env_configs\ep_000\clustered\moderate\arrivals_env_config.csv  (rows=8262)
[sav

<h1> Step 3: Agent Profiling </h1>

In this step, we construct a behavioral profile for each agent, capturing its local compute resources, task arrival rate, and the distribution of task types it generates. These profiles are later used for clustering agents and assigning suitable reinforcement learning strategies to each group.

In [29]:
# ---- Helper 1: per-agent per-slot arrival counts ----
def _per_agent_slot_counts(arrivals_df: pd.DataFrame) -> pd.DataFrame:
    """
    Count how many tasks each agent generates in each time slot.
    This is used to estimate lambda (arrival rate) statistics.
    """
    if not {"agent_id", "t_slot"}.issubset(arrivals_df.columns):
        raise ValueError("arrivals must contain 'agent_id' and 't_slot'.")
    grp = arrivals_df.groupby(["agent_id", "t_slot"], as_index=False).size()
    grp.rename(columns={"size": "count"}, inplace=True)
    return grp

# ---- Helper 2: estimate λ-mean and λ-variance per agent (tight dtypes) ----
def _lambda_stats_from_counts(counts_df: pd.DataFrame, Delta: float) -> pd.DataFrame:
    """
    Convert per-slot counts to rate statistics:
        lambda_mean = mean(count_per_slot) / Delta
        lambda_var  = var(count_per_slot)  / Delta^2
    """
    if counts_df.empty:
        return pd.DataFrame(columns=["agent_id", "lambda_mean", "lambda_var", "slots_observed"])

    agg = counts_df.groupby("agent_id")["count"].agg(
        lambda_mean_slot="mean",
        lambda_var_slot="var",
        slots_observed="count"
    ).reset_index()

    # If only one observation exists, variance becomes NaN → treat as zero.
    agg["lambda_var_slot"] = agg["lambda_var_slot"].fillna(0.0).astype(float)

    # Convert to per-second rates
    agg["lambda_mean"] = (agg["lambda_mean_slot"] / float(Delta)).astype(float)
    agg["lambda_var"]  = (agg["lambda_var_slot"]  / float(Delta**2)).astype(float)

    return agg[["agent_id", "lambda_mean", "lambda_var", "slots_observed"]]

# ---- Helper 3: task-type distribution per agent (robust + extra stats) ----
_TASK_TYPES = ["general", "latency_sensitive", "deadline_hard", "data_intensive", "compute_intensive"]

def _task_distribution_per_agent(tasks_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute distribution of task types per agent (probabilities sum to 1 for agents with tasks).
    Also adds light median features useful for clustering: b_mb_med, rho_med, mem_med, hard_share.
    """
    if not {"agent_id", "task_type"}.issubset(tasks_df.columns):
        raise ValueError("tasks must contain 'agent_id' and 'task_type'.")

    # Raw counts per (agent_id, task_type)
    cnt = tasks_df.groupby(["agent_id", "task_type"], as_index=False).size()
    piv = cnt.pivot(index="agent_id", columns="task_type", values="size").fillna(0.0)

    # Ensure all expected classes exist
    for t in _TASK_TYPES:
        if t not in piv.columns:
            piv[t] = 0.0

    # True count across all seen labels
    piv["n_tasks_agent"] = piv[_TASK_TYPES].sum(axis=1).astype(float)

    # Probabilities
    for t in _TASK_TYPES:
        piv[f"P_{t}"] = np.where(piv["n_tasks_agent"] > 0, piv[t] / piv["n_tasks_agent"], 0.0).astype(float)

    # Optional extra features for clustering (guard on availability)
    feats = {}
    have_feats = {"b_mb", "rho_cyc_per_mb", "mem_mb", "urgency"}
    if have_feats.issubset(tasks_df.columns):
        agg = tasks_df.groupby("agent_id").agg(
            b_mb_med=("b_mb", "median"),
            rho_med=("rho_cyc_per_mb", "median"),
            mem_med=("mem_mb", "median"),
            hard_share=("urgency", lambda s: float((s == "hard").mean()))
        ).reset_index()
        feats = agg.set_index("agent_id")

    # Join extra features (if any)
    piv = piv.join(feats, how="left")
    for c in ["b_mb_med", "rho_med", "mem_med", "hard_share"]:
        if c in piv.columns:
            piv[c] = piv[c].fillna(0.0).astype(float)
        else:
            piv[c] = 0.0

    # Probability mass sum (diagnostic)
    prob_cols = [f"P_{t}" for t in _TASK_TYPES]
    piv["TaskDist_sum"] = piv[prob_cols].sum(axis=1).astype(float)

    keep = ["n_tasks_agent", "TaskDist_sum", "b_mb_med", "rho_med", "mem_med", "hard_share"] + prob_cols
    return piv[keep].reset_index()

# ---- Helper 4: fraction of non-atomic (splittable) tasks ----
def _non_atomic_share_per_agent(tasks_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute the share of splittable (non-atomic) tasks per agent.
    """
    if not {"agent_id", "non_atomic"}.issubset(tasks_df.columns):
        # If missing, assume zero for all agents that exist in tasks
        agents = tasks_df.get("agent_id")
        if agents is None or len(agents) == 0:
            return pd.DataFrame(columns=["agent_id", "non_atomic_share"])
        return pd.DataFrame({"agent_id": agents.unique(), "non_atomic_share": 0.0})

    grp = tasks_df.groupby("agent_id")["non_atomic"].agg(
        non_atomic_share=lambda s: float((s == 1).mean())
    ).reset_index()
    return grp

# ---- Build agent profiles for ONE env_config ----
def build_agent_profiles_for_env(env_cfg: Dict[str, Any]) -> pd.DataFrame:
    """
    Construct per-agent profiles combining:
      - Local resource capacity (f_local, m_local, f_local_slot)
      - Arrival rate statistics (lambda_mean, lambda_var)
      - Task type distribution (P_general, P_latency_sensitive, P_deadline_hard, P_data_intensive, P_compute_intensive)
      - Splittability share (non_atomic_share)
      - MEC mapping if available (mec_id)
    """
    agents   = env_cfg["agents"].copy()
    arrivals = env_cfg["arrivals"]
    tasks    = env_cfg["tasks"]
    Delta    = float(env_cfg["Delta"])

    # Ensure cycles/slot exists
    if "f_local_slot" not in agents.columns and "f_local" in agents.columns:
        agents["f_local_slot"] = agents["f_local"].astype(float) * Delta

    # 1) Arrival statistics
    counts_df = _per_agent_slot_counts(arrivals)
    lam_df    = _lambda_stats_from_counts(counts_df, Delta=Delta)

    # 2) Task-type distribution (+ medians & hard_share)
    dist_df   = _task_distribution_per_agent(tasks)

    # 3) Splittable-task share
    na_df     = _non_atomic_share_per_agent(tasks)

    # 4) Agent→MEC mapping (optional)
    mec_map = None
    if "agent_to_mec" in env_cfg:
        a2m = env_cfg["agent_to_mec"]
        if isinstance(a2m, pd.Series):
            mec_map = a2m.rename("mec_id").reset_index()
            # if the index column name is lost, normalize it
            if mec_map.columns.tolist() == ["index", "mec_id"]:
                mec_map.rename(columns={"index": "agent_id"}, inplace=True)
        else:
            mec_map = pd.DataFrame({
                "agent_id": np.arange(len(a2m), dtype=int),
                "mec_id": np.asarray(a2m, dtype=int)
            })

    # Merge all components
    base = agents[["agent_id", "f_local", "f_local_slot", "m_local"]].copy()
    base[["f_local", "f_local_slot", "m_local"]] = base[["f_local", "f_local_slot", "m_local"]].astype(float)

    prof = (base
            .merge(lam_df,  on="agent_id", how="left")
            .merge(dist_df, on="agent_id", how="left")
            .merge(na_df,   on="agent_id", how="left"))

    if mec_map is not None:
        prof = prof.merge(mec_map, on="agent_id", how="left")

    # Fill missing for agents with no arrivals/tasks
    fill_zero = [
        "lambda_mean", "lambda_var", "slots_observed",
        "n_tasks_agent", "non_atomic_share",
        "TaskDist_sum", "b_mb_med", "rho_med", "mem_med", "hard_share"
    ] + [f"P_{t}" for t in _TASK_TYPES]
    for c in fill_zero:
        if c in prof.columns:
            prof[c] = prof[c].fillna(0.0).astype(float)

    # Soft warning if probabilities don't sum to ~1 for agents with tasks
    if "n_tasks_agent" in prof.columns and "TaskDist_sum" in prof.columns:
        mask = (prof["n_tasks_agent"] > 0) & (~np.isclose(prof["TaskDist_sum"], 1.0, atol=1e-6))
        if mask.any():
            n_bad = int(mask.sum())
            print(f"[warn] TaskDist_sum != 1.0 for {n_bad} agent(s). (tolerance 1e-6)")

    return prof

# ---- Batch profiling for ALL env_configs ----
def build_all_agent_profiles(env_configs: Dict[str, Dict[str, Dict[str, Any]]]):
    """
    Compute profiles for every (episode → topology → scenario) environment.
    Stores result both in return dict AND env_configs[...] for convenience.
    Output:
      profiles[ep_name][topology_name][scen_name] = DataFrame
    Also writes back to: env_configs[ep_name][topology_name][scen_name]["agent_profiles"]
    """
    out = {}
    for ep_name, by_topo in env_configs.items():
        out[ep_name] = {}
        for topo_name, by_scen in by_topo.items():
            out[ep_name][topo_name] = {}
            for scen_name, env_cfg in by_scen.items():
                prof = build_agent_profiles_for_env(env_cfg)
                out[ep_name][topo_name][scen_name] = prof
                env_cfg["agent_profiles"] = prof  # attach for direct access
    return out

In [30]:
# ---- Build + quick peek (optional) ----
agent_profiles = build_all_agent_profiles(env_configs)

# Example: Access the profile table for a specific episode / topology / scenario
print("\n ===EXAMPLE===")
display(agent_profiles["ep_000"]["clustered"]["heavy"].head())

# Alternatively, read directly from env_configs:
display(env_configs["ep_000"]["clustered"]["heavy"]["agent_profiles"].head())

,agent_id,f_local,f_local_slot,m_local,lambda_mean,lambda_var,slots_observed,n_tasks_agent,TaskDist_sum,b_mb_med,rho_med,mem_med,hard_share,P_general,P_latency_sensitive,P_deadline_hard,P_data_intensive,P_compute_intensive,non_atomic_share,mec_id
0,0,1.741183e+09,1.741183e+09,5713.849721,1.400325,0.474341,1846.0,2585.0,1.0,4.975150,1.496395e+09,64.062160,0.343907,0.182592,0.0,0.343907,0.093617,0.379884,0.467311,0
1,1,1.352326e+09,1.352326e+09,4566.428755,1.125348,0.140472,718.0,808.0,1.0,4.903463,1.542846e+09,64.020492,0.353960,0.169554,0.0,0.353960,0.092822,0.383663,0.433168,1
2,2,1.726668e+09,1.726668e+09,5815.120004,1.134063,0.138167,731.0,829.0,1.0,4.910049,1.540341e+09,66.006420,0.334138,0.184560,0.0,0.334138,0.086852,0.394451,0.434258,2
3,3,1.543616e+09,1.543616e+09,3539.850245,1.139918,0.147241,972.0,1108.0,1.0,5.067142,1.470254e+09,64.517043,0.371841,0.182310,0.0,0.371841,0.086643,0.359206,0.435921,3
4,4,1.130883e+09,1.130883e+09,4161.367769,1.277666,0.287951,1491.0,1905.0,1.0,5.031714,1.515057e+09,64.294464,0.344882,0.185302,0.0,0.344882,0.095538,0.374278,0.440945,4


,agent_id,f_local,f_local_slot,m_local,lambda_mean,lambda_var,slots_observed,n_tasks_agent,TaskDist_sum,b_mb_med,rho_med,mem_med,hard_share,P_general,P_latency_sensitive,P_deadline_hard,P_data_intensive,P_compute_intensive,non_atomic_share,mec_id
0,0,1.741183e+09,1.741183e+09,5713.849721,1.400325,0.474341,1846.0,2585.0,1.0,4.975150,1.496395e+09,64.062160,0.343907,0.182592,0.0,0.343907,0.093617,0.379884,0.467311,0
1,1,1.352326e+09,1.352326e+09,4566.428755,1.125348,0.140472,718.0,808.0,1.0,4.903463,1.542846e+09,64.020492,0.353960,0.169554,0.0,0.353960,0.092822,0.383663,0.433168,1
2,2,1.726668e+09,1.726668e+09,5815.120004,1.134063,0.138167,731.0,829.0,1.0,4.910049,1.540341e+09,66.006420,0.334138,0.184560,0.0,0.334138,0.086852,0.394451,0.434258,2
3,3,1.543616e+09,1.543616e+09,3539.850245,1.139918,0.147241,972.0,1108.0,1.0,5.067142,1.470254e+09,64.517043,0.371841,0.182310,0.0,0.371841,0.086643,0.359206,0.435921,3
4,4,1.130883e+09,1.130883e+09,4161.367769,1.277666,0.287951,1491.0,1905.0,1.0,5.031714,1.515057e+09,64.294464,0.344882,0.185302,0.0,0.344882,0.095538,0.374278,0.440945,4


<h1> Step 4: Clustering Agents </h1>

<h3> 4.1. Feature Matrix </h3>

To perform clustering, the characteristics of each agent must first be stored in a feature matrix. These characteristics include:

1) Local resources
2) task generation pattern

این باگ داره
برای همه اون کاری که باید و نمیکنه

In [43]:
# Step 1: Extract feature matrix for all agents across episodes, topologies, and scenarios
def build_agent_feature_matrix(agent_profiles):
    """
    Build the feature matrix for agents based on the columns available in agent_profiles.
    Iterates over all episodes, topologies, and scenarios to construct the matrix.
    """
    all_features = []

    for ep_name, by_topo in agent_profiles.items():
        for topo_name, by_scen in by_topo.items():
            for scen_name, agent_df in by_scen.items():
                # We want to extract the feature columns for each agent
                feature_columns = ['lambda_mean', 'lambda_var', 'P_general', 'P_latency_sensitive', 
                                   'P_deadline_hard', 'P_data_intensive', 'P_compute_intensive', 
                                   'non_atomic_share', 'f_local', 'm_local', 'b_mb_med', 'rho_med', 
                                   'mem_med', 'hard_share']
                # Ensure we are using the correct subset of columns and handle missing values
                agent_df = agent_df[feature_columns].fillna(0.0)  # Filling missing values with zero
                
                # Append the agent features to the list
                all_features.append(agent_df)

    # Concatenate all agent feature matrices into a single DataFrame
    feature_matrix = pd.concat(all_features, ignore_index=True)
    return feature_matrix

<h3> 4.2. Optimal Number of Clusters </h3>

To select the optimal number of clusters (K), we use a hybrid method that combines evaluation indices such as WCSS, Silhouette, DBI, and CH Index.

In [49]:
# Step 2: Select optimal K (number of clusters)
def compute_clustering_scores(feature_matrix, K_range):
    """
    Compute clustering scores (WCSS, Silhouette, DBI, CH) for a range of K values.
    This function returns a DataFrame with the scores for each K.
    """
    scores = []

    for K in K_range:
        kmeans = KMeans(n_clusters=K, random_state=42)
        kmeans.fit(feature_matrix)

        # Calculate WCSS (within-cluster sum of squares)
        wcss = np.sum((feature_matrix - kmeans.cluster_centers_[kmeans.labels_])**2)

        # Calculate Silhouette Score
        sil_score = silhouette_score(feature_matrix, kmeans.labels_)

        # Calculate Davies-Bouldin Index (DBI)
        dbi = davies_bouldin_score(feature_matrix, kmeans.labels_)

        # Calculate Calinski-Harabasz Index (CH)
        ch_score = calinski_harabasz_score(feature_matrix, kmeans.labels_)

        # Store the scores for this K
        scores.append({
            'K': K,
            'WCSS': wcss,
            'Silhouette': sil_score,
            'DBI': dbi,
            'CH': ch_score
        })

    scores_df = pd.DataFrame(scores)
    return scores_df

<h3> 4.3. Implementing K-Means Clustering </h3>

After selecting the optimal number of clusters (K_opt), we use the K-Means algorithm for clustering.

In [39]:
# Step 3: Run KMeans with the optimal number of clusters (K_opt)
def run_kmeans(feature_matrix, K_opt):
    """
    Run KMeans clustering with the optimal number of clusters (K_opt) on the feature matrix.
    """
    kmeans = KMeans(n_clusters=K_opt, random_state=42)
    kmeans.fit(feature_matrix)

    # Return the cluster labels and the cluster centers
    return kmeans.labels_, kmeans.cluster_centers_

<h3> 4.4. Store cluster labels </h3>

In [47]:
# Step 4: Store cluster labels in agent profiles
def store_cluster_labels_in_profiles(agent_profiles, labels, ep_name="ep_000", topo_name="clustered", scen_name="heavy"):
    """
    Store the cluster labels back in the agent profiles.
    """
    agent_profiles[ep_name][topo_name][scen_name]["cluster_labels"] = labels

Step-by-step implementation

In [51]:
# Build agent feature matrix for all agents across all environments
feature_matrix = build_agent_feature_matrix(agent_profiles)
feature_matrix

,lambda_mean,lambda_var,P_general,P_latency_sensitive,P_deadline_hard,P_data_intensive,P_compute_intensive,non_atomic_share,f_local,m_local,b_mb_med,rho_med,mem_med,hard_share
0,1.400325,0.474341,0.182592,0.0,0.343907,0.093617,0.379884,0.467311,1.741183e+09,5713.849721,4.975150,1.496395e+09,64.062160,0.343907
1,1.125348,0.140472,0.169554,0.0,0.353960,0.092822,0.383663,0.433168,1.352326e+09,4566.428755,4.903463,1.542846e+09,64.020492,0.353960
2,1.134063,0.138167,0.184560,0.0,0.334138,0.086852,0.394451,0.434258,1.726668e+09,5815.120004,4.910049,1.540341e+09,66.006420,0.334138
3,1.139918,0.147241,0.182310,0.0,0.371841,0.086643,0.359206,0.435921,1.543616e+09,3539.850245,5.067142,1.470254e+09,64.517043,0.371841
4,1.277666,0.287951,0.185302,0.0,0.344882,0.095538,0.374278,0.440945,1.130883e+09,4161.367769,5.031714,1.515057e+09,64.294464,0.344882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.106419,0.112175,0.232061,0.0,0.241221,0.114504,0.412214,0.316031,2.334762e+09,4156.023984,2.971737,1.209705e+09,64.873310,0.241221
158,1.077477,0.075214,0.232441,0.0,0.279264,0.107023,0.381271,0.357860,1.989451e+09,5301.096958,2.971286,1.207869e+09,63.704429,0.279264
159,1.052770,0.050118,0.208020,0.0,0.278195,0.112782,0.401003,0.401003,2.245627e+09,3973.902321,2.963276,1.188409e+09,66.241550,0.278195
160,1.058824,0.061137,0.182540,0.0,0.288360,0.084656,0.444444,0.333333,9.699633e+08,5954.812618,3.016342,1.233261e+09,67.249938,0.288360


In [52]:
# Compute clustering scores for a range of K values
K_range = range(2, 10)  # Testing K values from 2 to 9
scores_df = compute_clustering_scores(feature_matrix, K_range)
display(scores_df)

# Select optimal K (highest combined score)
K_opt = scores_df.loc[scores_df['Silhouette'].idxmax(), 'K']  # Select K with highest Silhouette score
print(K_opt)

c:\Users\niush\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\niush\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\niush\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\niush\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.

,K,WCSS,Silhouette,DBI,CH
0,2,lambda_mean 2.212910e+00 lambda_var...,0.503401,0.718692,242.256593
1,3,lambda_mean 1.890184e+00 lambda_var...,0.426744,0.877513,225.246795
2,4,lambda_mean 1.317319e+00 lambda_var...,0.454365,0.791397,220.105261
3,5,lambda_mean 6.699701e-01 lambda_var...,0.488596,0.693764,242.255108
4,6,lambda_mean 8.355879e-01 lambda_var...,0.462404,0.808251,253.707415
5,7,lambda_mean 8.908844e-01 lambda_var...,0.461836,0.788474,254.987462
6,8,lambda_mean 9.231353e-01 lambda_var...,0.472806,0.756262,253.348308
7,9,lambda_mean 8.686128e-01 lambda_var...,0.496988,0.745153,264.854731


2


In [50]:



# Run KMeans with the optimal K
labels, centers = run_kmeans(feature_matrix, K_opt)

# Store the cluster labels in the agent profiles
store_cluster_labels_in_profiles(agent_profiles, labels)

# Display the resulting cluster labels and centers
print("Cluster Labels:", labels)
print("Cluster Centers:", centers)

c:\Users\niush\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\niush\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\niush\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\niush\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.

ValueError: Length of values (162) does not match length of index (18)

In [40]:
# Step 2: Select optimal K (number of clusters)
K_range = range(2, 10)  # Testing K values from 2 to 9
scores_df = compute_clustering_scores(feature_matrix, K_range)

# Visualize the score table (optional)
print(scores_df)

# Step 3: Run KMeans with the optimal number of clusters (K_opt)
K_opt = scores_df.loc[scores_df['Score'].idxmax(), 'K']  # Select K with highest combined score
labels, centers = run_kmeans(feature_matrix, K_opt)

# Step 4: Construct profiles for each cluster center
cluster_centers_profiles = pd.DataFrame(centers, columns=feature_matrix.columns)
cluster_centers_profiles["cluster_label"] = range(K_opt)  # Add cluster label for each center

# Optional: Visualize the cluster centers
plt.scatter(feature_matrix['lambda_mean'], feature_matrix['lambda_var'], c=labels, cmap='viridis')
plt.scatter(cluster_centers_profiles['lambda_mean'], cluster_centers_profiles['lambda_var'], c='red', marker='x', label='Cluster Centers')
plt.xlabel('Lambda Mean')
plt.ylabel('Lambda Variance')
plt.title(f'Clustering with K={K_opt}')
plt.colorbar(label='Cluster')
plt.legend()
plt.show()

# Display the cluster centers
display(cluster_centers_profiles)

# Final output
display(agent_profiles.head())

# Save the centers profiles for later use
cluster_centers_profiles.to_csv('cluster_centers_profiles.csv', index=False)


TypeError: unhashable type: 'list'